<a href="https://colab.research.google.com/github/gretiere545/langdeck/blob/main/langdeck_sync.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Langdeck Synchro de bout en bout

In [2]:
#!/usr/bin/env python
# -*- coding: utf8 -*-
!pip install gspread --upgrade
# fix here => https://stackoverflow.com/questions/71347973/modulenotfounderror-no-module-named-gspread-models
!pip uninstall gspread-dataframe
!pip install gspread-dataframe
!pip install gspread-formatting
!pip install airtable-python-wrapper


import pandas as pd
import numpy as np
import requests
import json
import uuid
from datetime import datetime
import datetime
pd.set_option("display.width",1000)


# Ce bout de code pour pouvoir downloader des fichiers sur le drive
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Trad-Union/Corpus/ASAMLA
from google.colab import files

# Ce bout de code pour pouvoir downloader des google sheets dans des dataframes
from google.colab import auth
auth.authenticate_user()


#====== GSpread ================
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from gspread_formatting import *


#====== Google Credentials ================
from oauth2client.client import GoogleCredentials
# fix here => https://github.com/burnash/gspread/issues/1014
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)
#gc = gspread.authorize(GoogleCredentials.get_application_default())

#====== Airtable ================
from airtable import Airtable

!pip install 'lightning-flash[text]' --upgrade



#====== Deep Translate ================
!pip install transformers
!pip install txtai[pipeline] --upgrade
#!pip install sacremoses
#from txtai.pipeline import Translation
#translate = Translation()

!pip install -U deep-translator
!pip install --upgrade deepl


#====== MediaWiki ================
!pip install pymediawiki
# https://github.com/barrust/mediawiki
# https://pymediawiki.readthedocs.io/en/latest/code.html
from mediawiki import MediaWiki


#====== Dictionaries ================
# larousse API
!pip install larousse-api-sunbro

#====== Lemmatizers ================
# https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer
!pip install git+https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer.git
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
lemmatizer = FrenchLefffLemmatizer()

!export PYTHONIOENCODING=utf8


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gspread
    Found existing installation: gspread 3.4.2
    Uninstalling gspread-3.4.2:
      Successfully uninstalled gspread-3.4.2
Found existing installation: gspread-dataframe 3.0.8
Uninstalling gspread-dataframe-3.0.8:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/gspread_dataframe-3.0.8.dist-info/*
    /usr/local/lib/python3.7/dist-packages/gspread_dataframe.py
Proceed (y/n)? y
  Successfully uninstalled gspread-dataframe-3.0.8
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Mounted at /content/drive
/content/drive/MyDrive/Trad-Union/Corpus/ASAMLA
Looking in indexes: https://pypi.org/simple, https://us-

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 8.8 MB/s 
     |████████████████████████████████| 8.6 MB 46.1 MB/s 
     |████████████████████████████████| 13.1 MB 26.4 MB/s 
     |████████████████████████████████| 4.9 MB 37.0 MB/s 
     |████████████████████████████████| 509 kB 56.8 MB/s 
     |████████████████████████████████| 68 kB 6.1 MB/s 
     |████████████████████████████████| 296 kB 52.2 MB/s 
     |████████████████████████████████| 128 kB 35.1 MB/s 
     |████████████████████████████████| 308 kB 45.2 MB/s 
  Using cached pybind11-2.10.0-py3-none-any.whl (213 kB)
     |████████████████████████████████| 279 kB 50.3 MB/s 
     |████████████████████████████████| 46 kB 3.3 MB/s 
     |████████████████████████████████| 86 kB 6.0 MB/s 
     |████████████████████████████████| 83 kB 2.3 MB/s 

#### Ajout d'une langue


```
lang_dict = {
  'language': 'Chinois',
  'trigramme': 'cmn',
  'uri': '',
  'idx': 24,
  'update': 'false',
  'alphabet': '',
  'credits': '',
  'font-family': 'Adobe Arabic Regular',
  'font-path': '/content/drive/MyDrive/Trad-Union/Corpus/ASAMLA/resources/Adobe Arabic Regular.ttf',
  'text-direction': 'ltr',
  'glossary-subtitle': '',
  'font-family-bold': 'DejaVuSans-Bold',
  'unicode_substition': [],
  'meta': {'title-fr': '',
   'title-tr': '',
   'subtitle-fr': '',
   'subtitle-tr': '',
   'credits-fr': '',
   'credits-tr': ''},
  'wals_code': 'mnd'}
vk_languages.append(lang_dict)  
lang_config = 'med_vac_synthese.json'
set_cc_config (vk_languages, lang_config)
```



In [11]:
'''
lang_dict = {
  'language': 'Danois',
  'trigramme': 'dan',
  'uri': '',
  'idx': 29,
  'update': 'false',
  'alphabet': '',
  'credits': '',
  'font-family': '',
  'font-path': '',
  'text-direction': 'ltr',
  'glossary-subtitle': '',
  'font-family-bold': 'DejaVuSans-Bold',
  'unicode_substition': [],
  'meta': {'title-fr': '',
   'title-tr': '',
   'subtitle-fr': '',
   'subtitle-tr': '',
   'credits-fr': '',
   'credits-tr': ''},
  'wals_code': 'mnd'}
  '''

"\nlang_dict = {\n  'language': 'Danois',\n  'trigramme': 'dan',\n  'uri': '',\n  'idx': 29,\n  'update': 'false',\n  'alphabet': '',\n  'credits': '',\n  'font-family': '',\n  'font-path': '',\n  'text-direction': 'ltr',\n  'glossary-subtitle': '',\n  'font-family-bold': 'DejaVuSans-Bold',\n  'unicode_substition': [],\n  'meta': {'title-fr': '',\n   'title-tr': '',\n   'subtitle-fr': '',\n   'subtitle-tr': '',\n   'credits-fr': '',\n   'credits-tr': ''},\n  'wals_code': 'mnd'}\n  "

# Configuration

In [12]:
# IA
dict_vendor =[
    {"vendor-code":"11", "vendor-abbv":"neuml","vendor-name":"txtai(Neuml)","pivot":"en", "f":"fetch_deep_tlr_txtai"},
    {"vendor-code":"12", "vendor-abbv":"google","vendor-name":"GoogleTranslator","pivot":"en", "f":"fetch_deep_tlr_google"},
    {"vendor-code":"13", "vendor-abbv":"pons","vendor-name":"Pons","pivot":"fr", "f":"fetch_deep_tlr_pons"},   
    {"vendor-code":"14", "vendor-abbv":"deepl","vendor-name":"DeepL","pivot":"en", "f":"fetch_deep_tlr_deepl"}
]            

dict_lang_vendor = [{"language":"ams", "iso-code":"ar", "vendor-code":["12"]},
            {"language":"rus", "iso-code":"ru", "vendor-code":["12","14"]},
            {"language":"ukr", "iso-code":"uk", "vendor-code":["12"]},
            {"language":"rou", "iso-code":"ro", "vendor-code":["12","14"]},
            {"language":"alb", "iso-code":"sq", "vendor-code":["12"]},
            {"language":"eng", "iso-code":"en", "vendor-code":["12","14"]},
            {"language":"tur", "iso-code":"tr", "vendor-code":["12"]},
            {"language":"tig", "iso-code":"ti", "vendor-code":[]},
            {"language":"geo", "iso-code":"ka", "vendor-code":["12"]},
            {"language":"arm", "iso-code":"hy", "vendor-code":["12"]},
            {"language":"dar", "iso-code":"fa", "vendor-code":["12"]},
            {"language":"prs", "iso-code":"fa", "vendor-code":["12"]},
            {"language":"pst", "iso-code":"ps", "vendor-code":["12"]},
            {"language":"esp", "iso-code":"es", "vendor-code":["12","14"]},
            {"language":"pol", "iso-code":"pl", "vendor-code":["12","14"]},
            {"language":"som", "iso-code":"so", "vendor-code":["12"]},
            {"language":"amh", "iso-code":"am", "vendor-code":["12"]},
            {"language":"all", "iso-code":"de", "vendor-code":["12","14"]},
            {"language":"hun", "iso-code":"hu", "vendor-code":["12","14"]},
            {"language":"por", "iso-code":"pt", "vendor-code":["12"]},
            {"language":"aze", "iso-code":"az", "vendor-code":["12"]},
            {"language":"rnd", "iso-code":"rn", "vendor-code":[]},
            {"language":"kin", "iso-code":"rw", "vendor-code":["12"]},
            {"language":"cmn", "iso-code":"zh-CN", "vendor-code":["12"]},
            {"language":"urd", "iso-code":"ur", "vendor-code":["12"]},
            {"language":"hin", "iso-code":"hi", "vendor-code":["12"]},
            {"language":"ben", "iso-code":"bn", "vendor-code":["12"]},
            {"language":"bul", "iso-code":"bg", "vendor-code":["12"]},
            {"language":"dan", "iso-code":"da", "vendor-code":["12"]}]

from IPython.core.debugger import exception_colors
def get_cc_config(f):
  vk = []
  try:
    with open(f) as vk_f:
        vk = json.load(vk_f)
  except BaseException as e:
    print(e)
  return vk

def set_cc_config (vk, f):
  # sauvegarde du fichier de paramétrage
  with open(f, 'w') as fp:
      json.dump(vk, fp)  

#
# Ouverture de la Sheet Centrale (corpus_central_base)
#
def get_CCDB_wb(uri):
  sheet_central = uri
  wb_central = gc.open_by_url(sheet_central)
  return wb_central

def get_CCDB_data(wb, idx):
  t_corpus = wb.get_worksheet(idx)
  data_t_corpus = t_corpus.get_all_values()
  return data_t_corpus

#
# DataFrame CC global (toutes les langues)
#
def get_ccdf_global(CCDB):
  return pd.DataFrame(CCDB[1:], columns=CCDB[0])

#
# DataFrame CC Français (invariable)
#
def get_ccdf_fr(df_cc):
  # on supprime les doublons
  return df_cc[['uid','expression','theme','état','date','commentaires','index']]    

# Comparaison des versions de référentiels
def getLastModifiedTime (wb):
  # https://stackoverflow.com/questions/64254039/python-how-to-check-if-a-google-spreadsheets-was-updated
  # https://stackoverflow.com/questions/52260789/update-googlesheet-cell-with-timestamp-from-python
  revisions_uri = f'https://www.googleapis.com/drive/v3/files/{wb.id}/revisions'
  headers = {'Authorization': f'Bearer {GoogleCredentials.get_application_default().get_access_token().access_token}'}
  response = requests.get(revisions_uri, headers=headers).json()
  return response['revisions'][-1]['modifiedTime']    

def txtai_alpha (input, iso_code):
  try:
    translation = translate(input, iso_code)
  except Exception as e:
    translation = ""
    pass
  return translation, 2

def airtable_fetch_all_records():
  vk_at = airtable.get_all(view='Admin',sort='vocabulary_unit')
  df_vkat = pd.DataFrame.from_records((r['fields'] for r in vk_at))
  return df_vkat

def fetch_pivot_en (txt_fr):
  try:
    translation = translate(txt_fr, "en")
    print (translation)
  except Exception as e:
    translation = ""
    pass  
  return translation

def make_unique_id():
  return str(uuid.uuid4())[:8]  

def chk_uid (vk_uid, record):
  try:
    value = record['fields']['uid']
  except KeyError:
    # uid manquant
    print ("mssing uid")
    while True:
      auid = make_unique_id()
      if auid not in vk_uid:
        aDict = {}
        aDict["uid"] = auid
        print (record['id'] + " " + aDict["uid"])
        airtable.update(record['id'], aDict)
        vk_uid.append(auid) 
        break 
  finally:
    return

def airtable_check_missing_uids():
  vk_uid = df_vkat['uid'].tolist()
  for page in airtable.get_iter(view='Admin',sort='vocabulary_unit'):
    for record in page:
      chk_uid (vk_uid, record)

def compare_langdeck_time(hash, time):
  ts1 = time
  ts2 = df_langdeck.loc[df_langdeck["hash"]==hash]["unixtime"].values[0]
  return ts1 > ts2

def compare_airtable_time(hash, time):
  ts1 = time
  ts2 = df_airtable_for_langdeck.loc[df_airtable_for_langdeck["hash"]==hash]["unixtime"].values[0]
  return ts1 > ts2

def get_df_airtable_for_langdeck():
  df_vkat = airtable_fetch_all_records()
  df_temp = df_vkat.copy()
  #df_temp = df_temp[["uid","lemma","rank","vocabulary_unit","headword","pivot","pivot_en","alphabet","theme","timestamp","ams","eng","tur","rus","ukr","rou","hun","alb","geo","arm","dar","pst","prs","esp","pol","tig"]]
  df_temp = df_temp[["uid","lemma","rank","vocabulary_unit","headword","pivot","pivot_en","alphabet","theme","timestamp"]]
  for iso in dict_lang_vendor:
    # print (iso["language"])
    df_temp[iso["language"]] = np.nan


  
  #df_temp.rename(columns={"UID":"uid","Proposition":"expression","pivot_en":"pivot","idx_cor":"idx", "Maitre":"maitre","Ordre":"ordre","Thématique":"themes","lastModified":"timestamp"}, inplace=True)
  # https://stackoverflow.com/questions/28654047/convert-columns-into-rows-with-pandas
  df_airtable_for_langdeck = pd.melt(df_temp, id_vars=["uid", "lemma","rank","vocabulary_unit","headword","pivot","pivot_en","alphabet","timestamp","theme"], 
                    var_name="language", value_name="translation")
  #https://stackoverflow.com/questions/60393668/pandas-generate-unique-id-based-on-row-values
  #https://stackoverflow.com/questions/64604408/transform-dataframe-column-with-a-hash-value
  df_airtable_for_langdeck['hash'] = df_airtable_for_langdeck[['uid', 'language']].sum(axis=1).astype(str)
  #Fonction lambda pour récupérer le nom long de la langue (https://stackoverflow.com/questions/2191699/find-an-element-in-a-list-of-tuples)
  df_airtable_for_langdeck["language_fulltext"] = df_airtable_for_langdeck["language"].apply(lambda x:[item for item in [(a_dict["trigramme"],a_dict["language"]) for a_dict in vk_languages] if item[0] == x][0][1])  
  df_airtable_for_langdeck['timestamp'] = df_airtable_for_langdeck['timestamp'].astype('datetime64[s]')
  df_airtable_for_langdeck['unixtime'] = df_airtable_for_langdeck['timestamp'].astype('datetime64[s]').astype('int')
  # ajout des thématiques  
  # on éclate la liste des thèmes en 3 colonnes
  df_airtable_for_langdeck[["theme_1","theme_2","theme_3"]] = pd.DataFrame([pd.Series(x) for x in df_airtable_for_langdeck.theme])
  return df_airtable_for_langdeck   

def is_audio(f):
  path = "/content/drive/My Drive/appsheet/data/langdeck-1629444/"
  f = path + f
  result = True
  try:
    file = open(f, "r")
  except FileNotFoundError as e:
    # do nothing
    result = False
  return result     

def get_df_langdeck(url):
  # Ouverture de la Sheet Langdeck (mots à traduire)
  sheet = url
  wb  = gc.open_by_url(sheet)
  t_corpus = wb.worksheet("t_corpus_all_entries")
  data_t_corpus = t_corpus.get_all_values()
  df_langdeck = pd.DataFrame(data_t_corpus[1:], columns=data_t_corpus[0]).drop_duplicates()  

  df_langdeck['uid'] = df_langdeck['hash'].str[:8]

  df_langdeck['ordre'] = pd.to_numeric(df_langdeck['ordre'])
  df_langdeck['timestamp'] = df_langdeck['timestamp'].astype('datetime64[s]')
  df_langdeck['unixtime'] = df_langdeck['timestamp'].astype('datetime64[s]').astype('int')
  df_langdeck["translation_pivot_fr"] = df_langdeck["hash"].apply(lambda x:get_translation_pivot_fr(x))
  df_langdeck["translation_pivot_en"] = df_langdeck["hash"].apply(lambda x:get_translation_pivot_en(x))  
  #
  # df_langdeck[["translation_pivot_fr","translation_pivot_en"]] = df_langdeck["hash"].apply(lambda x:get_translation_pivot (x))
  #
  df_langdeck['is_audio'] = df_langdeck['audio'].apply(lambda x:is_audio(x))  
  return df_langdeck

def get_translation_pivot_fr (hash):
  r = ""
  try:
    r = df_airtable_for_langdeck.loc[df_airtable_for_langdeck["hash"]==hash]["pivot"].values.item()
  except Exception as e:
    #print (e)
    pass
  return r  

def get_translation_pivot_en (hash):
  r = ""
  try:
    r = df_airtable_for_langdeck.loc[df_airtable_for_langdeck["hash"]==hash]["pivot_en"].values.item()
  except Exception as e:
    #print (e)
    pass
  return r   

def get_translation_pivot (hash):
  piv_fr = ""
  piv_en = ""
  try:
    idx = df_airtable_for_langdeck.loc[df_airtable_for_langdeck["hash"]==hash].index
    piv_fr = df_airtable_for_langdeck.iloc[idx]["pivot"].values.item()
    piv_en = df_airtable_for_langdeck.iloc[idx]["pivot_en"].values.item()
  except Exception as e:
    #print (e)
    pass
  return piv_fr, piv_en

def get_wikipedia(text, language):
  wikipedia = MediaWiki(lang=language)  
  r = ""
  try:
    p = wikipedia.page(text)
    r = p.title + "\n" + p.summary
  except Exception as e:
    r = str(e)
    pass
  return r  


def get_larousse_lst (text):
  from larousse_api import larousse
  d = larousse.get_definitions(text)
  '''
  r = ""
  for i in d:
    i = i.replace("\t", "")
    i = i.replace(":\n", ": ")
    r = re.compile(r'^\n')
    i = r.sub("",i)
    r += str(i)
  '''
  return d

def get_larousse (text):
  from larousse_api import larousse
  vk = larousse.get_definitions(text)  
  s = ""
  for i in vk:
    s += i
  if s == "":
    s = "à renseigner"
  return s

In [13]:
# Config Langues
config_languages = 'med_vac_synthese.json'
vk_languages = get_cc_config(config_languages)

params = ()
api_key = "keyrI98TIqu6mbFcf" 
api_key_gmail = "keysgpJWq3aWwqSZe"
headers = {"Authorization": "Bearer " + api_key,"Content-Type" : "application/json"}
base_id = "appawT3gg7cJhPYv0" # Base Nouvelle HV
table_name = 'Corpus'

db_langdeck_url = "https://docs.google.com/spreadsheets/d/1iJLt3qAKq576eNirMwcsdRltg7nerwa9N70dA0vraa8/" #db-langdeck

# Chargement table Airtable
airtable = Airtable(base_id, table_name, api_key)
# Vers Dataframe
df_vkat = airtable_fetch_all_records()

# maj des uid manquants
airtable_check_missing_uids()

# Vérification qu'il n'existe pas des doublons (hash)
df_airtable_for_langdeck = get_df_airtable_for_langdeck()
df_airtable_for_langdeck[df_airtable_for_langdeck['hash'].duplicated()].sort_values("hash")
# on éclate la liste des thèmes en 3 colonnes
df_airtable_for_langdeck[["theme_1","theme_2","theme_3"]] = pd.DataFrame([pd.Series(x) for x in df_airtable_for_langdeck.theme])
# on ajoute une colonne de translation state (0=VOID; 1=PENDING; 2=OK; 3=KO)
df_airtable_for_langdeck["translation_state"] = df_airtable_for_langdeck["translation"].apply(lambda x:0 if pd.isna(x) else 1)

# ouverture base Langdeck (t_corpus_all_entries)
df_langdeck = get_df_langdeck(db_langdeck_url)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:169: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:198: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.


#### Ajouter des nouveaux enregistrements dans Langdeck

In [14]:
# suppression des enregistrements en double côté GSheet (supprimés dans AT)
df_langdeck.drop(df_langdeck[~df_langdeck["uid"].isin(df_airtable_for_langdeck["uid"])].index, inplace=True)
# DF des enregistrements dans AT et pas dans GS
df_new_in_airtable = df_airtable_for_langdeck[~df_airtable_for_langdeck["hash"].isin(df_langdeck["hash"])]
# on ajoute chaque enregistrement en passant par un DICT (pour mieux contrôler)
for index, row in df_new_in_airtable.iterrows():
  d = {}
  d["hash"]=row['hash']
  d["uid"]=row['uid']
  d["maitre"]=row["lemma"]
  d["ordre"]=row["rank"]
  d["expression"]=row["vocabulary_unit"]
  d["idx"]=row["alphabet"]
  d["timestamp"]=str(pd.to_datetime(row['timestamp']))
  d["language"]=row["language"]
  d["translation"]=row["translation"]
  d["language_fulltext"]=row["language_fulltext"]
  d["audio"]="Files_Files_/Corpus/t_expo_audio/" + row['uid'] + "-" + row['language'] + ".mp3"
  d["is_audio"]=False
  d["uploaded_file"]=""
  if pd.isna(row['theme_1']):
    row['theme_1'] = ""
  if pd.isna(row['theme_2']):
    row['theme_2'] = ""
  if pd.isna(row['theme_3']):
    row['theme_3'] = ""    
  d["theme_1"]=row['theme_1']
  d["theme_2"]=row['theme_2']
  d["theme_3"]=row['theme_3']    
  d["theme_1_svg"]=""
  d["theme_2_svg"]=""
  d["theme_3_svg"]=""
  d["unixtime"]=str(pd.to_datetime(row['timestamp']))
  d["translation_state"]=0
  d["translation_pivot"]=row["pivot_en"]
  d["translation_ai_source"]=""     
  d["translation_score"]=0
  d["translation_state_txt"]=""
  d["translation_pivot_fr"]=row["pivot"]
  d["translation_pivot_en"]=row["pivot_en"]
  # ajout dans le DF
  df_langdeck = df_langdeck.append(d, ignore_index = True)

#### Mise à jour des méta données dans Langdeck

In [15]:
def f_upload_meta(hash):
  try:
    idx = df_airtable_for_langdeck.loc[df_airtable_for_langdeck["hash"]==hash].index
    r1 = df_airtable_for_langdeck.iloc[idx]["theme_1"].values.item()
    r2 = df_airtable_for_langdeck.iloc[idx]["theme_2"].values.item()
    r3 = df_airtable_for_langdeck.iloc[idx]["theme_3"].values.item()    
    r4 = df_airtable_for_langdeck.iloc[idx]["lemma"].values.item()    
    r5 = df_airtable_for_langdeck.iloc[idx]["rank"].values.item()    
    r6 = df_airtable_for_langdeck.iloc[idx]["vocabulary_unit"].values.item()    
    r7 = df_airtable_for_langdeck.iloc[idx]["alphabet"].values.item()    
    r8 = df_airtable_for_langdeck.iloc[idx]["pivot"].values.item()    
    r9 = df_airtable_for_langdeck.iloc[idx]["pivot_en"].values.item()    
  except Exception as e:        
    pass  
  return pd.Series([r1, r2, r3, r4, r5, r6, r7, r8, r9])

In [16]:
df_langdeck[["theme_1","theme_2","theme_3","maitre","ordre","expression","idx","translation_pivot_fr","translation_pivot_en"]] = df_langdeck["hash"].apply(lambda x:f_upload_meta(x))

#### Mise à jour des traductions dans Airtable (en attente)
```
for index, row in df_newly_updated_in_langdeck.iterrows():
  if row["uid"] is not None:
    record = airtable.match('uid', row["uid"])
    if bool(record) is True:
      aDict = {}
      aDict[row["language"]] = row["translation"]
      airtable.update(record['id'], aDict)
      print ("Updating status for " + row["uid"])
```



In [17]:
try:
  wb  = gc.open_by_url(db_langdeck_url)
  t_corpus = wb.worksheet("t_corpus_all_entries")
  wb.del_worksheet(t_corpus)
except:
  print ("Onglet inexistant !")
  pass
wb.add_worksheet("t_corpus_all_entries", 1, 1)
export_sheet = wb.worksheet("t_corpus_all_entries")
set_with_dataframe(export_sheet, df_langdeck)

# Table Corpus Headword (Lemme)
Indexé sur le Lemme

In [18]:
# table vue Lemme
at_lemme = Airtable(base_id, "Corpus", api_key)
vk_lemme = at_lemme.get_all(view='Lemme',sort=['alphabet', 'rank','vocabulary_unit'])
df_vklemme = pd.DataFrame.from_records((r['fields'] for r in vk_lemme))
df_vklemme.drop_duplicates(subset=['vocabulary_unit'], inplace=True)

In [19]:
### on charge la table t_corpus_head
sheet = db_langdeck_url
wb  = gc.open_by_url(sheet)
tbl = wb.worksheet("t_corpus_head")
# on recharge avec les nouvelles données
data = tbl.get_all_values()
df_corpus_head = pd.DataFrame(data[1:], columns=data[0]).drop_duplicates(subset=['terme_pref']) 
# différence dans les deux sens
# df_diff = df_langdeck[~df_langdeck["maitre"].isin(df_corpus_head["terme_pref"])].drop_duplicates(subset=['maitre'])
# on vire ce qui n'est plus dans le ref aiertabel
df_corpus_head.drop(df_corpus_head[~df_corpus_head["terme_pref"].isin(df_langdeck["maitre"])].index, inplace=True) 
df_head_lines = df_vklemme[(df_vklemme["vocabulary_unit"].isin(df_corpus_head["terme_pref"])) & (df_vklemme["headword"]==True)]

In [20]:
def f_content_head(lemma):
  df_f = df_vklemme.loc[df_vklemme["lemma"]==lemma].sort_values(by=['rank'])
  definition = df_corpus_head.loc[df_corpus_head["terme_pref"]==lemma]["definition"].values.item()
  if len(definition) == 0:
    print("Appel Larousse " + lemma)
    definition = get_larousse(lemma)
    if len(definition) == 0:
      definition = "à renseigner"
  t = ""
  for index, row in df_f.iterrows():
    strng=""
    for i in row['category']:
      strng +=str(i)    
    if row['rank'] == 1:
      t = t + "<b>"  + row["vocabulary_unit"] + "</b><br />"
      t = t + "<i>" +strng + "</i>&#9;"
      t = t + definition + "<br />"
      t += "<br />"
    elif row['rank'] == 2:
      t = t + "&#9670;&nbsp;"  + row["vocabulary_unit"] + "<br />"
    else:
      t = t + "&#9671;&nbsp;"  + row["vocabulary_unit"] + "<br />"  
  return t

def f_update_head(lemma):
  #print(lemma)
  r1 = df_head_lines.loc[df_head_lines["vocabulary_unit"]==lemma]["uid"].values.item()
  r2 = df_head_lines.loc[df_head_lines["vocabulary_unit"]==lemma]["pivot_en"].values.item()
  r3 = f_content_head(lemma)

  return pd.Series([r1, r2, r3])

### Append : Ajout des nouvelles lignes Head

In [21]:
# DF des enregistrements dans AT et pas dans GS
df_head_newlines = df_vklemme[(~df_vklemme["vocabulary_unit"].isin(df_corpus_head["terme_pref"])) & (df_vklemme["headword"]==True)]
# on ajoute chaque enregistrement en passant par un DICT (pour mieux contrôler)
for index, row in df_head_newlines.iterrows():
  d = {}
  d["uid"]=row['uid']
  d["terme_pref"]=row["lemma"]
  d["idx"]=row["alphabet"]
  d["translation_pivot"]=row["pivot"]  
  d["translation_pivot_en"]=row["pivot_en"]    
  d["wikipedia_fr"]=""
  d["wikipedia_en"]=""
  d["wiki_search_text_fr"]=""
  d["wiki_search_text_en"]=""
  d["wikipedia_fr_state"]=0
  d["wikipedia_en_state"]=0 
  d["definition"] = get_larousse(row["lemma"])
  d["content"]= ""
  # ajout dans le DF
  df_corpus_head = df_corpus_head.append(d, ignore_index = True)

# apprès ajout, il faut recharger ce DF
df_head_lines = df_vklemme[(df_vklemme["vocabulary_unit"].isin(df_corpus_head["terme_pref"])) & (df_vklemme["headword"]==True)]  

### Update

#### Appel Larousse si non renseigné
https://github.com/quentin-dev/larousse_api

https://pypi.org/project/PyDictionary/


In [22]:
df_corpus_head["definition"] = df_corpus_head[["terme_pref","definition"]].apply(lambda x:get_larousse(x[0]) if x[1]=="" else x[1], axis=1)

#### Mide à jour des lignes existantes Head

In [23]:
df_corpus_head[["uid","translation_pivot_en","content"]] = df_corpus_head["terme_pref"].apply(lambda x:f_update_head(x))

#### Appel MediaWiki (suspendu)


```
df_corpus_head["wikipedia_fr"] = df_corpus_head[["terme_pref","wikipedia_fr"]].apply(lambda x:get_wikipedia(x[0], "fr") if pd.isna(x[1]) else x[1], axis=1)
```



#### Appels Wiki Head

Différences entre Langdeck REf et termes_pref

In [24]:
# traduction en anglais (pivot)
from google.colab import output
output.enable_custom_widget_manager()



In [25]:
'''
df_corpus_head["wiki_search_text_fr"] = df_corpus_head["translation_pivot"]
df_corpus_head["wiki_search_text_en"] = df_corpus_head["translation_pivot_en"]

df_corpus_head["wikipedia_fr"] = df_corpus_head[["translation_pivot","wikipedia_fr"]].apply(lambda x:get_wikipedia(x[0], "fr") if pd.isna(x[1]) else x[1], axis=1)
df_corpus_head["wikipedia_en"] = df_corpus_head[["translation_pivot_en","wikipedia_en"]].apply(lambda x:get_wikipedia(x[0], "en") if pd.isna(x[1]) else x[1], axis=1)

df_corpus_head["wikipedia_fr_state"]=df_corpus_head["wikipedia_fr"].apply(lambda x: 1 if (len(x)>0 and x[0]=="\n") else 0)
df_corpus_head["wikipedia_en_state"]=df_corpus_head["wikipedia_en"].apply(lambda x: 1 if (len(x)>0 and x[0]=="\n") else 0)
'''

'\ndf_corpus_head["wiki_search_text_fr"] = df_corpus_head["translation_pivot"]\ndf_corpus_head["wiki_search_text_en"] = df_corpus_head["translation_pivot_en"]\n\ndf_corpus_head["wikipedia_fr"] = df_corpus_head[["translation_pivot","wikipedia_fr"]].apply(lambda x:get_wikipedia(x[0], "fr") if pd.isna(x[1]) else x[1], axis=1)\ndf_corpus_head["wikipedia_en"] = df_corpus_head[["translation_pivot_en","wikipedia_en"]].apply(lambda x:get_wikipedia(x[0], "en") if pd.isna(x[1]) else x[1], axis=1)\n\ndf_corpus_head["wikipedia_fr_state"]=df_corpus_head["wikipedia_fr"].apply(lambda x: 1 if (len(x)>0 and x[0]=="\n") else 0)\ndf_corpus_head["wikipedia_en_state"]=df_corpus_head["wikipedia_en"].apply(lambda x: 1 if (len(x)>0 and x[0]=="\n") else 0)\n'

In [26]:
try:
  t_ = wb.worksheet("t_corpus_head")
  wb.del_worksheet(t_)
except:
  print ("Onglet inexistant !")

wb.add_worksheet("t_corpus_head", 1, 1)
export_sheet = wb.worksheet("t_corpus_head")
set_with_dataframe(export_sheet, df_corpus_head)

In [27]:
from google.colab import output
output.disable_custom_widget_manager()

## Vocabulaire

### On recherche les unités de vocabulaire et les headwords dans AT, et si non trouvés, on les créé
On cherche dans le DF Langdeck car s'ils s'y trouvent, ils sont aussi dans AT
Si pas trouvé on crée dans AT (ils seront créés dans Langdeck ultérieurement (processus))

In [28]:
def f_get_vocab_unit (s):
  s = lemmatizer.lemmatize(s).lower()
  vu = ""
  try:
    vu = df_langdeck.loc[df_langdeck["expression"].str.lower()==s]["expression"].drop_duplicates().values.item()
    print("Found " + vu + " for " + s)
  except Exception as e:
    vu = vu.capitalize()
    print ("Unité de vocabulaire / Headword / Lemme introuvable : " + s)
    # création dans le ref
    pass
  return vu

In [29]:
def f_get_headword (s):
  s = lemmatizer.lemmatize(s).lower()
  hw = ""
  try:
    hw = df_langdeck.loc[df_langdeck["expression"].str.lower()==s]["maitre"].drop_duplicates().values.item()
    print("Found " + hw + " for " + s)
  except Exception as e:
    hw = hw.capitalize()
    print ("Unité de vocabulaire / Headword / Lemme introuvable : " + s.capitalize())
    # création dans le ref
    pass
  return hw

In [30]:
tbl = wb.worksheet("t_ec_vocabulary")
data = tbl.get_all_values()
df_vocab = pd.DataFrame(data[1:], columns=data[0])
df_vocab["headword"]=df_vocab["headword"].map(lambda x: x.capitalize())
df_vocab["Vocabulaire"]=df_vocab["Vocabulaire"].map(lambda x: x.capitalize())
df_vocab["b_hw"]=df_vocab["headword"].apply(lambda x:f_get_headword(x))
df_vocab["b_vu"]=df_vocab["Vocabulaire"].apply(lambda x:f_get_vocab_unit(x))

Found Asile for asile
Found Asile for asile
Found Cancer for cancer
Found Famille for famille
Found Oui for oui
Found Enfant for enfant
Found Bonjour for bonjour
Found Dossier for dossier
Found Cancer for cancer
Found Diabète for diabète
Found Insuffisance for insuffisance
Found Chimiothérapie for chimiothérapie
Found Opération for opération
Found Comment for comment
Found Gynécologie for gynécologie
Found Cancer for cancer
Found Médecine for médecine
Found Asile for asile
Found Ovaire for ovaire
Found Prise en charge for prise en charge
Found Médicament for médicament
Found Tension for tension
Found Bilan for bilan
Found Assistance for assistance
Found Chimiothérapie for chimiothérapie
Found Utérus for utérus
Found Ovaire for ovaire
Found Cardiaque for cardiaque
Found Consultation for consultation
Found Consultation for consultation
Found Douleur for douleur
Found Demande d’asile for demande d’asile
Found Vous avez fait une demande d’asile ? for vous avez fait une demande d’asile ?
Fo

### Ajout des Lemmes (Headword)

In [31]:
df_vocab

,PKID,EC_KeyFR,Vocabulaire,headword,b_hw,b_vu
0,EC03-VOC-001,EC03-FRA-001_15,Demande d’asile,Asile,Asile,Demande d’asile
1,EC03-VOC-002,EC03-FRA-001_15,Vous avez fait une demande d’asile ?,Asile,Asile,Vous avez fait une demande d’asile ?
2,EC03-VOC-003,EC03-FRA-001_18,Cancer,Cancer,Cancer,Cancer
3,EC03-VOC-004,EC03-FRA-001_11,Famille,Famille,Famille,Famille
4,9b29a399,EC03-FRA-001_14,Oui,Oui,Oui,Oui
5,21fa910f,EC03-FRA-001_13,Enfant,Enfant,Enfant,Enfant
6,4d57e8bc,EC03-FRA-001_01,Bonjour,Bonjour,Bonjour,Bonjour
7,d17dac94,EC03-FRA-001_02,Dossier,Dossier,Dossier,Dossier
8,2a3d6d31,EC03-FRA-001_28,Cancer,Cancer,Cancer,Cancer
9,e708a1fc,EC03-FRA-001_28,Diabète,Diabète,Diabète,Diabète


In [32]:
for index, row in df_vocab.iterrows():
  # si b_hw est vide : on créé la ligne
  if row["b_hw"] == "":
    print (row["headword"])
    record = at_lemme.match('lemma', row["headword"])
    if record:
      print ("Found lemma : " + row["headword"] + " " + record['fields']['lemma'])
    else:
      # on crée une nouvelle ligne
      fields = {"lemma": row["headword"], "alphabet": row["headword"][0], "headword":True, "pending":True}
      at_lemme.insert(fields, typecast=True)
      print ("New lemma :" + str(fields))      

### Ajout des Unités de vocabulaire

In [33]:
for index, row in df_vocab.iterrows():
  # si b_hw est vide : on créé la ligne
  if row["b_vu"] == "":
    print (row["Vocabulaire"])
    record = at_lemme.match('vocabulary_unit', row["Vocabulaire"])
    if record:
      print ("Found vocabulary_unit : " + row["Vocabulaire"] + " " + record['fields']['vocabulary_unit'])
    else:
      # on crée une nouvelle ligne
      fields = {"vocabulary_unit":row["Vocabulaire"], "lemma": row["headword"], "alphabet": row["headword"][0], "headword":False, "pending":True}
      at_lemme.insert(fields, typecast=True)
      print ("New vocabulary_unit :" + str(fields))   

In [34]:
# On recharge si besoin la table vue Lemme
at_lemme = Airtable(base_id, "Corpus", api_key)
vk_lemme = at_lemme.get_all(view='Lemme',sort=['alphabet', 'rank','vocabulary_unit'])
df_vklemme = pd.DataFrame.from_records((r['fields'] for r in vk_lemme))
df_vklemme.drop_duplicates(subset=['vocabulary_unit'], inplace=True)

In [35]:
# on enregistre dans GSheet
df_vocab.drop(['b_hw', 'b_vu'], axis=1, inplace=True)
try:
  t_ = wb.worksheet("t_ec_vocabulary")
  wb.del_worksheet(t_)
except:
  print ("Onglet inexistant !")

wb.add_worksheet("t_ec_vocabulary", 1, 1)
export_sheet = wb.worksheet("t_ec_vocabulary")
set_with_dataframe(export_sheet, df_vocab)

#Table Traduction Corpus IA


```
language="rou"
iso_code = df_lang_vendor.loc[df_lang_vendor["language"]=="rou"]["iso-code"].values.item()

df_lang_vendor.loc[df_lang_vendor["language"]=="rou"]["iso-code"].values.item()

from deep_translator import GoogleTranslator
input="Bonjour"
translation = GoogleTranslator(source='auto', target="zh-CN").translate(input)
translation

# from deep_translator import DeeplTranslator
text = 'Quand avez-vous pris vos médicaments ?'
translated = DeeplTranslator(api_key="ea9751e0-a036-d65e-14af-3169e96b11e4:fx", source="fr", target=iso_code, use_free_api=True).translate(text)
translated
```



In [36]:
df_lang_vendor = pd.DataFrame(dict_lang_vendor)

In [37]:
def translation_google (input, language, language_filter, iso_code, ai_code,langue_pivot):
  from deep_translator import GoogleTranslator
  translation = input
  state = 0 #par défaut
  if language == language_filter:
    translation = ""
    try:
      translation = GoogleTranslator(source=langue_pivot, target=iso_code).translate(input)
      if translation != "":
        print (input + "("+ langue_pivot +") "+ " : " + translation)
        state = 0 #found
    except Exception as e:
      print (e)
      translation = ""
      pass
  else:
    ai_code = 0
  return translation, state, ai_code



def translation_deepl (input, language, language_filter, iso_code, ai_code,langue_pivot):
  from deep_translator import DeeplTranslator
  translation = input
  state = 0 #par défaut

  if language == language_filter:
    translation = ""
    try:
      translation = DeeplTranslator(api_key="ea9751e0-a036-d65e-14af-3169e96b11e4:fx", source=langue_pivot, target=iso_code, use_free_api=True).translate(input)
      if translation != "":
        print (input + "("+ langue_pivot +") "+ " : " + translation)
        state = 0 #found
    except Exception as e:
      print (e)
      translation = ""
      pass
  else:
    ai_code = 0

  return translation, state, ai_code


def translation_pons (input, language, language_filter, iso_code, ai_code):

  translation = input
  state = 0 #par défaut

  if language == language_filter:
    translation = ""
    try:
      translation = PonsTranslator(source='fr', target=iso_code).translate(input)
      if translation != "":
        print (input + " : " + translation)
        state = 0 #found
    except Exception as e:
      translation = ""
      pass
  else:
    ai_code = 0
  return translation, state, ai_code

def translation_txtai (input, language, language_filter, iso_code, ai_cod):

  translation = input
  state = 0 #par défaut

  if language == language_filter:
    translation = ""
    try:
      translation = translate(input, iso_code)
      if translation != "":
        print (input + " : " + translation)
        state = 0 #found
    except Exception as e:
      translation = ""
      pass
  else:
    ai_code = 0
  return translation, state, ai_code


1. On charge la base des traductions

### Appel de l'IA Google (12)

In [38]:
def call_nlp_google (language, iso_code, vc, langue_pivot, df_langdeck_ai_temp):
  from deep_translator import GoogleTranslator
  if len(df_langdeck_ai_temp) > 0:
    df_langdeck_ai_temp[["translation_ai","translation_state","translation_ai_source"]] = df_langdeck_ai_temp[["translation_pivot","expression","language","translation_state"]].apply(
        lambda x:translation_google(x[0],x[2],language, iso_code, vc, langue_pivot), 
        axis=1, 
        result_type="expand")
  df_google = df_langdeck_ai_temp.copy()   
  return df_google   

def call_nlp_deepl (language, iso_code, vc, langue_pivot, df_langdeck_ai_temp):
  from deep_translator import DeeplTranslator
  if len(df_langdeck_ai_temp) > 0:
    df_langdeck_ai_temp[["translation_ai","translation_state","translation_ai_source"]] = df_langdeck_ai_temp[["translation_pivot","expression","language","translation_state"]].apply(
        lambda x:translation_deepl(x[0],x[2], language, iso_code, vc, langue_pivot), 
        axis=1, 
        result_type="expand")
  df_deepl = df_langdeck_ai_temp.copy()
  return df_deepl

def call_nlp_language_vendor (language, iso_code, vc, langue_pivot):
  # dataframe temporaire pour chaque appel IA
  df_langdeck_ai_temp = df_langdeck.copy()
  df_langdeck_ai_temp["translation_pivot"]=df_langdeck_ai_temp[["translation_pivot_fr","translation_pivot_en"]].apply(lambda x:x[0] if langue_pivot=="fr" else x[1] , axis=1)  
  # ajout d'un uid par concaténation des 3 clés
  df_langdeck_ai_temp["uid"] = df_langdeck_ai_temp["hash"].map(str) + vc 
  df_langdeck_ai_temp = df_langdeck_ai_temp [["uid","hash","maitre","expression","language","translation","translation_pivot","translation_state","translation_ai_source"]]
  df_langdeck_ai_temp["translation_ai"]=""
  df_langdeck_ai_temp["translation_state"]=2
  df_langdeck_ai_temp["tr_pivot_lang"]=langue_pivot

  # on exclut les entrées déjà renseignées par l'iA
  df_langdeck_ai_temp = df_langdeck_ai_temp.loc[(~df_langdeck_ai_temp["uid"].isin(df_corpus_nlp["uid"])) & (df_langdeck_ai_temp["language"]==language)]
  # on appelle la traduction IA si :
  # - le uid n'existe pas dans la DF IA df_corpus_nlp
  # - le filtre est positionné  
  #df_langdeck_ai_temp = df_langdeck_ai_temp.head()
  for dic in dict_vendor:
    if dic["vendor-code"]==vc:
      if dic["vendor-abbv"]=="google":
        df_google = call_nlp_google (language, iso_code, vc, langue_pivot, df_langdeck_ai_temp)
        df_return = df_google
      elif dic["vendor-abbv"]=="deepl":
        df_deepl = call_nlp_deepl (language, iso_code, vc, langue_pivot, df_langdeck_ai_temp)
        df_return = df_deepl
  return df_return    

In [39]:
sheet = db_langdeck_url
wb  = gc.open_by_url(sheet)
t_corpus = wb.worksheet("t_corpus_nlp")
data_t_corpus = t_corpus.get_all_values()
# on recharge avec les nouvelles données
df_corpus_nlp =  pd.DataFrame(data_t_corpus[1:], columns=data_t_corpus[0]).drop_duplicates()  
df_corpus_nlp = df_corpus_nlp[["uid","hash","maitre","expression","language","translation","translation_pivot","tr_pivot_lang","translation_state","translation_ai_source","translation_ai"]]	

df_langdeck_ai_temp = df_langdeck.copy()
FILTER_PIVOT = "en"
df_langdeck_ai_temp["translation_pivot"]=df_langdeck_ai_temp[["translation_pivot_fr","translation_pivot_en"]].apply(lambda x:x[0] if FILTER_PIVOT=="fr" else x[1] , axis=1)

df_langdeck_ai_temp.loc[pd.isna(df_langdeck_ai_temp["translation_pivot"])]

,hash,uid,maitre,ordre,expression,idx,timestamp,language,translation,language_fulltext,...,theme_2_svg,theme_3_svg,unixtime,translation_state,translation_pivot,translation_ai_source,translation_score,translation_state_txt,translation_pivot_fr,translation_pivot_en


In [40]:
# concaténation des dataframes
frames =[]
frames.append(df_corpus_nlp)

# on fait une passe par langue et par langue pivot
# liste complète : 
vk_filter = list(map(lambda d: d["trigramme"], vk_languages))
for FILTER_LANG in vk_filter:
  for dic in dict_lang_vendor:
    language = dic["language"]
    iso_code = dic["iso-code"]
    # Si la langue est celle du filtre ?
    if language == FILTER_LANG:
      # Pour chaque code vendor qui gère la langue 
      for vc in dic["vendor-code"]:
        df_return = call_nlp_language_vendor (FILTER_LANG, iso_code, vc, FILTER_PIVOT)
        frames.append(df_return)



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

With an empty stomach(en)  : Med tom mave
At what time of day do you take the medicine?(en)  : På hvilket tidspunkt på dagen tager du medicinen?
Abscesses(en)  : Bylder
Abdomen(en)  : Mave
Concomitant febrile attack in an other family member ?(en)  : Samtidig feberanfald hos et andet familiemedlem?
Accident(en)  : Ulykke
Brain accident(en)  : Hjerneulykke
Childbirth(en)  : Fødsel
To give birth(en)  : At føde
Tinnitus(en)  : Tinnitus
Age(en)  : Alder
Alcohol(en)  : Alkohol
Allergy(en)  : Allergi
Drug allergy(en)  : Lægemiddelallergi
Food allergy(en)  : Fødevareallergi
Drug allergy(en)  : Lægemiddelallergi
To lie down(en)  : At ligge ned
Lie down!(en)  : Ligge ned!
AME- State Medical Aid(en)  : AME-Statens lægehjælp
Analgesic(en)  : Smertestillende
Urine analysis(en)  : Urinanalyse
Urine analysis(en)  : Urinanalyse
Anesthesia(en)  : Anæstesi
General anesthesia(en)  : Generel anæstesi
Local anesthesia(en)  : Lokalbedøvelse
Angina(en)  : Angina
Antecedents(en)  : Forhistorier
Anti inflamma

In [41]:
# quand tout a été itéré, on concatène    
df_langdeck_ai = pd.concat(frames)
print (len(df_langdeck_ai))
# on uniformise le type
df_langdeck_ai['translation_ai_source'] = df_langdeck_ai['translation_ai_source'].astype(str)
# on supprime les doublons
df_langdeck_ai = df_langdeck_ai.drop_duplicates(subset=['uid'], keep='last')
# on supprime les codes ai == 0
df_langdeck_ai = df_langdeck_ai.loc[df_langdeck_ai["translation_ai_source"]!="0"]
# par défaut état = 0
df_langdeck_ai["translation_state"] = df_langdeck_ai["translation_state"].map(lambda x:x if int(x)!=0 else 0)


15629


In [42]:
# on enregistre dans GSheet
try:
  t_data = wb.worksheet("t_corpus_nlp")
  wb.del_worksheet(t_data)
except:
  print ("Onglet inexistant !")

wb.add_worksheet("t_corpus_nlp", 1, 1)
export_sheet = wb.worksheet("t_corpus_nlp")
set_with_dataframe(export_sheet, df_langdeck_ai)

# on positionne cette variable si une traduction pat l'AI est disponible
df_langdeck["translation_ai_source"]=df_langdeck["hash"].apply(lambda x: len(df_langdeck_ai.loc[df_langdeck_ai["hash"]==x]))

# on enregistre dans GSheet
try:
  t_corpus = wb.worksheet("t_corpus_all_entries")
  wb.del_worksheet(t_corpus)
except:
  print ("Onglet inexistant !")
  pass
wb.add_worksheet("t_corpus_all_entries", 1, 1)
export_sheet = wb.worksheet("t_corpus_all_entries")
set_with_dataframe(export_sheet, df_langdeck)

### Vue Lexique Thématique
*Pour cette vue, on récupère le DF Langdeck et on extrait la liste des thèmes (l'ensemble des valeurs dans Thème1, 2, 3)
Ensuite, on ajoute autant de colonnes que d'éléments de la liste
Enfin, on supprime les colonnes Theme1,2,3 et on utilise pd.melt pour transposer les colonnes de thème en valeurs. On suprrme les themes = False*

In [43]:
# on crée une liste des valeurs uniques des 3 colonnes de thèmes
frames = [df_langdeck["theme_1"],df_langdeck["theme_2"],df_langdeck["theme_3"]]
vk_theme = list(set(pd.concat(frames).to_list()))
vk_theme = [x for x in vk_theme if x == x]

# on ajoute les colonnes à partir de la liste et on valorise T/F
df_thema = df_langdeck[["hash","maitre","ordre","expression","idx","language","language_fulltext","translation","theme_1","theme_2","theme_3"]].copy()
df_thema = pd.concat([df_thema, pd.DataFrame(columns = vk_theme)])
for item in vk_theme:
  df_thema[item] = df_thema[["theme_1","theme_2","theme_3"]].apply(lambda x:True if (x[0]==item or x[1]==item or x[2]==item) else False, axis=1)

df_thema = df_thema.drop(["theme_1","theme_2","theme_3"], axis=1)
df_thema = pd.melt(df_thema, id_vars=["hash","maitre","ordre","expression","idx","language","language_fulltext","translation"], 
                    var_name="thema", value_name="is_thema")
df_thema = df_thema.loc[df_thema["is_thema"]]

# on enregistre dans GSheet
try:
  t_corpus = wb.worksheet("t_lex_thema")
  wb.del_worksheet(t_corpus)
except:
  print ("Onglet inexistant !")
  pass
wb.add_worksheet("t_lex_thema", 1, 1)
export_sheet = wb.worksheet("t_lex_thema")
set_with_dataframe(export_sheet, df_thema)


### Traductions IA pour les EDC

On charge la table t_ec_content_trad et on créé une table des propositions  comme t_corpus_nlp

In [44]:

'''
df_ec_trad["translation_pivot_fr"]=pd.NA
df_ec_trad["translation_pivot_en"]=pd.NA
df_ec_trad["translation_pivot"]=pd.NA
df_ec_trad["translation_ai_source"]=0
df_ec_trad["translation_ai"]=pd.NA


trad2_temp = wb.worksheet("t_ec_content_trad2")
data = trad2_temp.get_all_values()
df_ec_trad2 = pd.DataFrame(data[1:], columns=data[0])
'''

'\ndf_ec_trad["translation_pivot_fr"]=pd.NA\ndf_ec_trad["translation_pivot_en"]=pd.NA\ndf_ec_trad["translation_pivot"]=pd.NA\ndf_ec_trad["translation_ai_source"]=0\ndf_ec_trad["translation_ai"]=pd.NA\n\n\ntrad2_temp = wb.worksheet("t_ec_content_trad2")\ndata = trad2_temp.get_all_values()\ndf_ec_trad2 = pd.DataFrame(data[1:], columns=data[0])\n'

In [45]:
tbl = wb.worksheet("t_ec_content_trad")
data = tbl.get_all_values()
df_ec_trad = pd.DataFrame(data[1:], columns=data[0])

tbl = wb.worksheet("t_ec_content_fr")
data = tbl.get_all_values()
df_ec_fr = pd.DataFrame(data[1:], columns=data[0])
df_ec_fr["translation_pivot_fr"]=df_ec_fr["PhraseFR"]
df_ec_fr["translation_pivot_en"]=pd.NA
df_ec_fr["translation_pivot"]=df_ec_fr[["translation_pivot_fr","translation_pivot_en"]].apply(lambda x:x[0] if FILTER_PIVOT=="fr" else x[1] , axis=1)

tbl = wb.worksheet("t_ec_meta")
data = tbl.get_all_values()
df_ec_meta = pd.DataFrame(data[1:], columns=data[0])

### Ajout d'une langue si besoin

In [46]:
FILTER_PIVOT = "fr"
FILTER_LANG="dan"
#FILTER_EC = "EC01"
# on créé une table des propositions comme t_corpus_nlp
df_ec_nlp = df_langdeck_ai.copy()
df_ec_nlp.drop(df_ec_nlp.index,inplace=True) 
# clé EDC+TRAD
df_ec_fr["id_trad"]=df_ec_fr["ID"].apply(lambda x:x[:5] + FILTER_LANG.upper() + "-" + x[9:])
# nombre de trad IA disponibles pour chaque clé
df_ec_fr["translation_ai_source"]=df_ec_fr["id_trad"].apply(lambda x:len(df_ec_nlp.loc[df_ec_nlp["uid"]==x]))
df_ec_fr["PhraseTR"]=pd.NA
df_ec_fr["AudioTR"]=pd.NA
df_ec_fr["AudioTR_AI"]=pd.NA
df_ec_fr.rename(columns={"ID":"EC_KeyFR","id_trad":"ID"}, inplace=True)
'''
df_ec_fr = pd.merge(df_ec_fr, df_ec_trad2[["ID","translation_ai"]], how="left", left_on="id_trad", right_on="ID")
df_ec_fr.drop('ID_y', axis=1, inplace=True)
df_ec_fr.rename(columns={"ID_x":"EC_KeyFR","id_trad":"ID"}, inplace=True)
'''

'\ndf_ec_fr = pd.merge(df_ec_fr, df_ec_trad2[["ID","translation_ai"]], how="left", left_on="id_trad", right_on="ID")\ndf_ec_fr.drop(\'ID_y\', axis=1, inplace=True)\ndf_ec_fr.rename(columns={"ID_x":"EC_KeyFR","id_trad":"ID"}, inplace=True)\n'

#### On a dans A toutes les EC en Fr et dans B les traductions existantes en RUS
On complète les traductions manquantes par appel IA

In [47]:
# DF des enregistrements dans t_ec_content_fr et pas dans t_ec_content_trad
df_newlines = df_ec_fr[~df_ec_fr["ID"].isin(df_ec_trad["ID"])]

# on ajoute les lignes absentes dans TRD
frames =[]
frames.append(df_ec_trad)
frames.append(df_newlines)
df_ec_trad = pd.concat(frames)

In [48]:
len(df_newlines)

152

In [49]:
'''
df_ec_trad.drop('translation_ai', axis=1, inplace=True)
df_ec_trad = pd.merge(df_ec_trad, df_ec_fr[["ID","translation_ai"]], how="left", on="ID")
'''

'\ndf_ec_trad.drop(\'translation_ai\', axis=1, inplace=True)\ndf_ec_trad = pd.merge(df_ec_trad, df_ec_fr[["ID","translation_ai"]], how="left", on="ID")\n'

In [50]:
def simpleGoogleTranslate (txt, pivot, iso_code):
  from deep_translator import GoogleTranslator
  try:
    r = GoogleTranslator(source=pivot, target=iso_code).translate(txt)
    print (r)
  except Exception as e:
    print (e)
    pass
  return r

In [51]:
# on met à jour les traductions IA
iso_code = df_lang_vendor.loc[df_lang_vendor["language"]==FILTER_LANG]["iso-code"].values.item()
df_ec_trad["EC_Key"]=df_ec_trad["ID"].apply(lambda x:x[:8])
df_ec_trad["translation_ai"] = df_ec_trad[["PhraseFR","translation_ai","EC_Key"]].apply(
        lambda x:simpleGoogleTranslate(x[0], FILTER_PIVOT, iso_code) if ((pd.isna(x[1]) or x[1]=="") and x[2][5:].lower()==FILTER_LANG) else x[1], 
        axis=1)

Hej, jeg er læge S. og jeg er ledsaget af en medicinstuderende.
Da det er første gang, vi mødes, vil jeg stille dig et par spørgsmål for at forberede din fil.
Er du fra Georgia? fra Rusland?
Fra Rusland
Hvornår ankom du til Frankrig?
I november 2021
Hvordan kom du?
Med bus, via Grækenland
Blev du længe i Grækenland?
12 dage.
Kom du alene eller sammen med andre familiemedlemmer?
Jeg gik for at finde hende.
Bor du sammen med dine børn?
Ja
Har du søgt asyl?
Ingen. Jeg har en tid den 9. februar hos en socialrådgiver i området.
Bliver du fulgt for specifikke helbredsproblemer?
Hun har kræft.
Nok til at ?
Gynækologi. Hendes livmoder og æggestokke blev fjernet.
Så det er kræft i æggestokkene.
Det var hvornår?
6. august 2021. Efter operationen havde hun smerter. Hendes søn var med hende. I september tog hun med min datter og i november nåede vi at komme til Frankrig.
Har hun fået kemoterapi?
Hun startede med kemo, men fik det kun tre gange efter operationen.
Kender du navnet på kemo?
Nix
Udove

In [52]:
# si la traduction est absente en PhraseTR => On colle la traduction IA
df_ec_trad["PhraseTR"] = df_ec_trad[["PhraseTR","translation_ai"]].apply(lambda x:x[1] if (pd.isna(x[0]) or x[0]=="") else x[0], axis=1)
f_path = "Files_Files_/assets/audio/"
f_path_ai = "Files_Files_/assets/audio/tts/"
df_ec_trad["translation_pivot"] = df_ec_trad[["translation_pivot","PhraseFR"]].apply(lambda x: x[1] if (pd.isna(x[0]) or x[0]=="") else x[0], axis=1)

In [53]:
'''
df_ec_trad["AudioFR"] = df_ec_trad[["AudioFR","EC_KeyFR"]].apply(lambda x: f_path + x[1] + ".mp3" if (pd.isna(x[0]) or x[0]=="") else x[0], axis=1)
df_ec_trad["AudioTR"] = df_ec_trad[["AudioTR","ID"]].apply(lambda x: f_path + x[1] + ".mp3" if (pd.isna(x[0]) or x[0]=="") else x[0], axis=1)
df_ec_trad["AudioTR_AI"] = df_ec_trad[["AudioTR_AI","ID"]].apply(lambda x: f_path_ai + x[1] + ".mp3" if (pd.isna(x[0]) or x[0]=="") else x[0], axis=1)
'''

'\ndf_ec_trad["AudioFR"] = df_ec_trad[["AudioFR","EC_KeyFR"]].apply(lambda x: f_path + x[1] + ".mp3" if (pd.isna(x[0]) or x[0]=="") else x[0], axis=1)\ndf_ec_trad["AudioTR"] = df_ec_trad[["AudioTR","ID"]].apply(lambda x: f_path + x[1] + ".mp3" if (pd.isna(x[0]) or x[0]=="") else x[0], axis=1)\ndf_ec_trad["AudioTR_AI"] = df_ec_trad[["AudioTR_AI","ID"]].apply(lambda x: f_path_ai + x[1] + ".mp3" if (pd.isna(x[0]) or x[0]=="") else x[0], axis=1)\n'

In [54]:
df_ec_trad["AudioFR"] = df_ec_trad[["AudioFR","EC_KeyFR"]].apply(lambda x: f_path + x[1] + ".mp3", axis=1)
df_ec_trad["AudioTR"] = df_ec_trad[["AudioTR","ID"]].apply(lambda x: f_path + x[1] + ".mp3" , axis=1)
df_ec_trad["AudioTR_AI"] = df_ec_trad[["AudioTR_AI","ID"]].apply(lambda x: f_path_ai + x[1] + ".mp3", axis=1)

# on enregistre dans GSheet
try:
  t_ = wb.worksheet("t_ec_content_trad")
  wb.del_worksheet(t_)
except:
  print ("Onglet inexistant !")
  pass
wb.add_worksheet("t_ec_content_trad", 1, 1)
export_sheet = wb.worksheet("t_ec_content_trad")
set_with_dataframe(export_sheet, df_ec_trad)

# Il faut mettre à jour la table de correspondance des EC t_ec_bylang
df_ec_index = pd.DataFrame()
df_ec_index["ID"]=df_ec_trad["EC_Key"].drop_duplicates()

# Il faut mettre à jour la table de correspondance des EC t_ec_bylang
df_ec_index = pd.DataFrame()
df_ec_index["ID"]=df_ec_trad["EC_Key"].drop_duplicates()
df_ec_index["EC_Key"]=df_ec_index["ID"].map(lambda x:x[:4])
df_ec_index = pd.merge(df_ec_index, df_ec_meta[["ID","Nom"]], how="left", left_on="EC_Key", right_on="ID")
df_ec_index.rename(columns={"ID_x":"ID","Nom":"EC_Desc"}, inplace=True)
df_ec_index.drop('ID_y', axis=1, inplace=True)
df_ec_index["Language_code"]=df_ec_index["ID"].map(lambda x:x[5:].lower())
df_ec_index["code_2d"]=df_ec_index["Language_code"].map(lambda x: df_lang_vendor.loc[df_lang_vendor["language"]==x]["iso-code"].values.item()) 
df_ec_index["Language"] = df_ec_index["Language_code"].map(lambda x: list(filter(lambda i: i['trigramme'] == x, vk_languages))[0]["language"])
df_ec_index["Interpret"] = pd.NA
df_ec_index = pd.merge(df_ec_index, df_ec_meta[["ID","Cover"]], how="left", left_on="EC_Key", right_on="ID")
df_ec_index.drop('ID_y', axis=1, inplace=True)
df_ec_index.rename(columns={"ID_x":"ID","Nom":"EC_Desc"}, inplace=True)

# on enregistre dans GSheet
try:
  t_corpus = wb.worksheet("t_ec_bylang")
  wb.del_worksheet(t_corpus)
except:
  print ("Onglet inexistant !")
  pass
wb.add_worksheet("t_ec_bylang", 1, 1)
export_sheet = wb.worksheet("t_ec_bylang")
set_with_dataframe(export_sheet, df_ec_index)

# TTS

In [55]:
import os
!pip install gtts --upgrade
from gtts import gTTS
# !gtts-cli --all

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [56]:
def text2speech (text, language, path, slow):
  # https://www.geeksforgeeks.org/convert-text-speech-python/
  # Passing the text and language to the engine, 
  # here we have marked slow=False. Which tells 
  # the module that the converted audio should 
  # have a high speed
  r = True
  try:
    myobj = gTTS(text=text, lang=language, slow=slow)
    myobj.save(path)
    print (path + "(" + language + ") : okay")
  except Exception as e:
    print (language + " - " + str(e))
    r = False
    if os.path.getsize(path) == 0:
      try:
        os.remove(path)
        print (path + " file removed.")
      except OSError:
        pass    
    
  return r


In [57]:
'''
def text2speech (text, language, path):
  # https://www.geeksforgeeks.org/convert-text-speech-python/
  # Passing the text and language to the engine, 
  # here we have marked slow=False. Which tells 
  # the module that the converted audio should 
  # have a high speed
  r = False
  try:
    myobj = gTTS(text=text, lang=language, slow=False)
    myobj.save(path)
    print (path)
    r = True
  except Exception as e:
    print(str(e))

  return r

'''

'\ndef text2speech (text, language, path):\n  # https://www.geeksforgeeks.org/convert-text-speech-python/\n  # Passing the text and language to the engine, \n  # here we have marked slow=False. Which tells \n  # the module that the converted audio should \n  # have a high speed\n  r = False\n  try:\n    myobj = gTTS(text=text, lang=language, slow=False)\n    myobj.save(path)\n    print (path)\n    r = True\n  except Exception as e:\n    print(str(e))\n\n  return r\n\n'

In [58]:
vk_filter = list(map(lambda d: d["trigramme"], vk_languages))

In [59]:
for langue in vk_filter:
  iso_code = df_lang_vendor.loc[df_lang_vendor["language"]==langue]["iso-code"].values.item()
  print (langue + ":" + iso_code)

ams:ar
eng:en
tur:tr
rus:ru
ukr:uk
rou:ro
hun:hu
tig:ti
alb:sq
geo:ka
arm:hy
dar:fa
pst:ps
prs:fa
aze:az
esp:es
amh:am
all:de
pol:pl
som:so
rnd:rn
kin:rw
por:pt
cmn:zh-CN
hin:hi
ben:bn
urd:ur
bul:bg
dan:da


### Version langue choisie

In [53]:
is_audio('/content/drive/My Drive/appsheet/data/langdeck-1629444/Files_Files_/assets/audio/tts/EC03-PRS-001_01.mp3')

False

In [60]:
audio_path = "/content/drive/My Drive/appsheet/data/langdeck-1629444/"
filtre = ["dan"]
force = False
slow = False

for langue in filtre:
  iso_code = df_lang_vendor.loc[df_lang_vendor["language"]==langue]["iso-code"].values.item()
  LANG_FILTER = langue
  df_ec_trad_filtered = df_ec_trad.copy()
  df_ec_trad_filtered["language"]=df_ec_trad_filtered["EC_Key"].map(lambda x:x[5:].lower())
  df_ec_trad_filtered = df_ec_trad_filtered.loc[df_ec_trad_filtered["language"]==LANG_FILTER]
  if len(df_ec_trad_filtered) > 0:
    df_ec_trad_filtered[["PhraseFR","EC_Key","AudioFR"]].apply(lambda x:text2speech(x[0], "fr", audio_path + x[2], False) if is_audio(x[2])==False else False, axis=1)
    df_ec_trad_filtered[["PhraseTR","EC_Key","AudioTR_AI"]].apply(lambda x:text2speech(
        x[0], 
        df_lang_vendor.loc[df_lang_vendor["language"]==x[1][5:].lower()]["iso-code"].values.item(), 
        audio_path + x[2],
        slow) 
        if (is_audio(x[2])==False or force==True) else False, axis=1)    

/content/drive/My Drive/appsheet/data/langdeck-1629444/Files_Files_/assets/audio/tts/EC03-DAN-001_01.mp3(da) : okay
/content/drive/My Drive/appsheet/data/langdeck-1629444/Files_Files_/assets/audio/tts/EC03-DAN-001_02.mp3(da) : okay
/content/drive/My Drive/appsheet/data/langdeck-1629444/Files_Files_/assets/audio/tts/EC03-DAN-001_03.mp3(da) : okay
/content/drive/My Drive/appsheet/data/langdeck-1629444/Files_Files_/assets/audio/tts/EC03-DAN-001_04.mp3(da) : okay
/content/drive/My Drive/appsheet/data/langdeck-1629444/Files_Files_/assets/audio/tts/EC03-DAN-001_05.mp3(da) : okay
/content/drive/My Drive/appsheet/data/langdeck-1629444/Files_Files_/assets/audio/tts/EC03-DAN-001_06.mp3(da) : okay
/content/drive/My Drive/appsheet/data/langdeck-1629444/Files_Files_/assets/audio/tts/EC03-DAN-001_07.mp3(da) : okay
/content/drive/My Drive/appsheet/data/langdeck-1629444/Files_Files_/assets/audio/tts/EC03-DAN-001_08.mp3(da) : okay
/content/drive/My Drive/appsheet/data/langdeck-1629444/Files_Files_/asse

### Version tt langues

In [ ]:
audio_path = "/content/drive/My Drive/appsheet/data/langdeck-1629444/"
for langue in vk_filter:
  iso_code = df_lang_vendor.loc[df_lang_vendor["language"]==langue]["iso-code"].values.item()
  LANG_FILTER = langue
  df_ec_trad_filtered = df_ec_trad.copy()
  df_ec_trad_filtered["language"]=df_ec_trad_filtered["EC_Key"].map(lambda x:x[5:].lower())
  df_ec_trad_filtered = df_ec_trad_filtered.loc[df_ec_trad_filtered["language"]==LANG_FILTER]
  if len(df_ec_trad_filtered) > 0:
    df_ec_trad_filtered[["PhraseFR","EC_Key","AudioFR"]].apply(lambda x:text2speech(x[0], "fr", audio_path + x[2]) if is_audio(x[2])==False else False, axis=1)
    df_ec_trad_filtered[["PhraseTR","EC_Key","AudioTR_AI"]].apply(lambda x:text2speech(
        x[0], 
        df_lang_vendor.loc[df_lang_vendor["language"]==x[1][5:].lower()]["iso-code"].values.item(), 
        audio_path + x[2],
        False) 
        if is_audio(x[2])==False else False, axis=1)    

TypeError: ignored

# Ajout d'une EC depuis Airtable

In [61]:
tbl = wb.worksheet("t_ec_meta")
data = tbl.get_all_values()
df_ec_meta = pd.DataFrame(data[1:], columns=data[0])

In [62]:
# On recharge si besoin la table vue Lemme
# Chargement table Airtable
base_id_ec = "appi8taq2HcbmWyLo" # Base Nouvelle HV
table_name_ec = 'Etudes de cas'
table_name_location = 'Lieux'
table_name_service = 'Services'
table_name_interpret = 'Interprètes'
table_name_language = 'Langues'
# table Lieux
airtable_location = Airtable(base_id_ec, table_name_location, api_key)
vk_location = airtable_location.get_all(view='Grid view',sort=['Lieu'])
df_vk_location = pd.DataFrame.from_records((r['fields'] for r in vk_location))
# table Services
airtable_service = Airtable(base_id_ec, table_name_service, api_key)
vk_service = airtable_service.get_all(view='Grid view',sort=['Name'])
df_vk_service = pd.DataFrame.from_records((r['fields'] for r in vk_service))
# table Interprètes
airtable_interpret = Airtable(base_id_ec, table_name_interpret, api_key)
vk_interpret = airtable_interpret.get_all(view='Grid view',sort=['Interprète'])
df_vk_interpret = pd.DataFrame.from_records((r['fields'] for r in vk_interpret))
# table Langues
airtable_language = Airtable(base_id_ec, table_name_language, api_key)
vk_language = airtable_language.get_all(view='Grid view',sort=['Name'])
df_vk_language = pd.DataFrame.from_records((r['fields'] for r in vk_language))

airtable_ec = Airtable(base_id_ec, table_name_ec, api_key)
vk_ec = airtable_ec.get_all(view='EC',sort=['uid', 'ec_date'])
df_vk_ec = pd.DataFrame.from_records((r['fields'] for r in vk_ec))
df_vk_ec.drop_duplicates(subset=['ec_name'], inplace=True)
# on filtre
df_vk_ec = df_vk_ec.loc[df_vk_ec["ec_status"]=="Réalisé"]

#df_vk_ec = df_vk_ec.loc[df_vk_ec["uid"]==21]

# on ajoute les colonnes pour le merge avec ec_meta
df_vk_ec["uid"]=df_vk_ec["uid"].astype(int)
df_vk_ec["ID"]=df_vk_ec["uid"].apply(lambda x:"EC"+ "{:02d}".format(x))

# on réserve pour la suite (script CR)
df_ec_script = df_vk_ec[["ec_name", "uid","ec_script"]]

# on ne conserve que les ec absentes dans meta
df_vk_ec = df_vk_ec.loc[~df_vk_ec["ID"].isin(df_ec_meta["ID"])]
# on convertit les listes à 1 élément en string
df_vk_ec["ec_service"] = df_vk_ec["ec_service"].apply(lambda x: ''.join([str(i) for i in x]))
# on complète les infos
df_vk_ec["Thème"] = df_vk_ec["ec_service"].apply(lambda x: airtable_service.get(x)['fields']['Name'])
df_vk_ec["Domaine"] = df_vk_ec["ec_service"].apply(lambda x: airtable_service.get(x)['fields']['Domaine'])
df_vk_ec["Présentation"] = df_vk_ec["ec_service"].apply(lambda x: airtable_service.get(x)['fields']['Resume'])
# interprète
df_vk_ec["ec_interpret"] = df_vk_ec["ec_interpret"].apply(lambda x: ''.join([str(i) for i in x]))
df_vk_ec["Interprète"] = df_vk_ec["ec_interpret"].apply(lambda x: airtable_interpret.get(x)['fields']['Nom complet'])
# lieu (location)
df_vk_ec["ec_location"] = df_vk_ec["ec_location"].apply(lambda x: ''.join([str(i) for i in x]))
df_vk_ec["Lieu"] = df_vk_ec["ec_location"].apply(lambda x: airtable_location.get(x)['fields']['Adresse'])
# Langue
df_vk_ec["ec_langue"] = df_vk_ec["ec_langue"].apply(lambda x: ''.join([str(i) for i in x]))
df_vk_ec["Language"] = df_vk_ec["ec_langue"].apply(lambda x: airtable_language.get(x)['fields']['Name'])
df_vk_ec["Language_code"] = df_vk_ec["ec_langue"].apply(lambda x: airtable_language.get(x)['fields']['Code'])

# on supprime les colonnes devenues inutiles
df_vk_ec.drop(["ec_service","ec_interpret","interpret_nom_complet","service_resume","ec_location","location_adress","ec_langue","ec_domaine","ec_script","Notes","Attachments"], axis=1, inplace=True)

df_vk_ec.rename ({"ec_date":"Date","ec_status":"Statut","ec_name":"Nom","ec_doc_url":"CR","ec_contexte":"Contexte"}, axis=1, inplace=True)

df_vk_ec["Cover"]=df_vk_ec["ID"].apply(lambda x: "Files_Files_/assets/images/" + x + ".png")
df_vk_ec["Audio1"]=pd.NA
df_vk_ec["Audio2"]=pd.NA
df_vk_ec.drop(["uid"], axis=1, inplace=True)

frames=[]
frames.append(df_ec_meta)
frames.append(df_vk_ec)
df_ec_meta_update = pd.concat(frames)



In [63]:
df_ec_meta_update

,ID,Nom,Domaine,Thème,Language_code,Language,Date,Lieu,Interprète,Statut,Cover,CR,Audio1,Audio2,Présentation,Contexte
0,EC01,Etude de cas n°1,Médical,Hepato-Gastroentérologie,rus,Russe,16 décembre 2021 14:15,CHU,Nina,Réalisé,Files_Files_/assets/images/EC01.png,https://drive.google.com/open?authuser=0&id=1c...,,,,
1,EC02,Etude de cas n°2,Médical,Vaccination,rom,Roumain,4 janvier 2022 10:30,PASS Consultation Jean-Guillon,Alexandra,Réalisé,Files_Files_/assets/images/EC02.png,https://drive.google.com/open?authuser=0&id=1P...,,,,
2,EC03,Etude de cas n°3,Médical,PASS,rus,Russe,7 janvier 2022 08:45,PASS Consultation Jean-Guillon\nBâtiment Le To...,Nina,Réalisé,Files_Files_/assets/images/EC03.png,https://drive.google.com/open?authuser=0&id=1Q...,,,La permanence d'accès aux soins de santé (Pass...,La patient est une femme de nationalité russe....
3,EC04,Etude de cas n°4,Médical,PASS,rom,Roumain,28 décembre 2021 17:28,PASS Consultation Jean-Guillon\nBâtiment Le To...,,Réalisé,Files_Files_/assets/images/EC04.png,https://drive.google.com/open?authuser=0&id=1x...,,,,
4,EC05,Etude de cas n°5,Médical,Vaccination,rom,Roumain,28 décembre 2021 17:31,PASS Consultation Jean-Guillon,,Réalisé,Files_Files_/assets/images/EC05.png,https://drive.google.com/open?authuser=0&id=1S...,,,,
5,EC06,Etude de cas n°6,Médical,Pédiatrie,tur,Turc,26 janvier 2022 11:30,CHU,Gulseren,Réalisé,Files_Files_/assets/images/EC06.png,https://docs.google.com/document/d/1cwEgb2QoZq...,,,,
6,EC07,Etude de cas n°7,Médical,PASS,tur,Turc,2 février 2022 14:15,PASS Consultation Jean-Guillon,Canan,Réalisé,Files_Files_/assets/images/EC07.png,https://docs.google.com/document/d/1oLOmvPwCh5...,,,,
7,EC08,Etude de cas n°8,Médical,Infectiologie,ams,Arabe (Moderne Standard),15 février 2022 09:30,CHU,Wafa,Réalisé,Files_Files_/assets/images/EC08.png,https://docs.google.com/document/d/1NtTtUkIVJv...,,,,
8,EC09,Etude de cas n°9,Médical,Orthopédie,dri,Dari,4 mars 2022 14:24,CHU,Massouda,Réalisé,Files_Files_/assets/images/EC09.png,https://docs.google.com/document/d/1x4vKkLryaP...,,,,
9,EC10,Etude de cas n°10,Médical,Hémophilie,hun,Hongrois,22 mars 2022 11:30,CHU,Annamaria,Réalisé,Files_Files_/assets/images/EC10.png,https://docs.google.com/document/d/1nWFkcB1n4s...,,,,


In [64]:
# on enregistre dans GSheet
try:
  t_corpus = wb.worksheet("t_ec_meta")
  wb.del_worksheet(t_corpus)
except:
  print ("Onglet inexistant !")
  pass
wb.add_worksheet("t_ec_meta", 1, 1)
export_sheet = wb.worksheet("t_ec_meta")
set_with_dataframe(export_sheet, df_ec_meta_update)

## EC : scripts

In [65]:
table_name_script = 'EC_Script'
# table Lieux
airtable_script = Airtable(base_id_ec, table_name_script, api_key)
vk_script = airtable_script.get_all(view='Grid view',sort=['sequence','uid'])
df_vk_script = pd.DataFrame.from_records((r['fields'] for r in vk_script))

tbl = wb.worksheet("t_ec_meta")
data = tbl.get_all_values()
df_ec_meta = pd.DataFrame(data[1:], columns=data[0])

tbl = wb.worksheet("t_ec_content_fr")
data = tbl.get_all_values()
df_ec_script_content = pd.DataFrame(data[1:], columns=data[0])
df_ec_script_content_new = df_ec_script_content.drop(df_ec_script_content.index)

df_ec_script["EC_Key"] = df_ec_script["uid"].apply(lambda x:"EC"+ "{:02d}".format(x))

# on ne conserve que les ec scripts absentes dans meta ET avec du contenu
df_ec_script = df_ec_script.loc[(~df_ec_script["EC_Key"].isin(df_ec_script_content["EC_Key"])) & (~pd.isna(df_ec_script["ec_script"]))]

if len(df_ec_script) > 0:

  # on explose (transpose) les ids de dialogues
  df_ec_script = df_ec_script.explode('ec_script')

  df_ec_script["PhraseFR"] = df_ec_script["ec_script"].apply(lambda x: airtable_script.get(x)['fields']['texte'])
  df_ec_script["Locuteur"] = df_ec_script["ec_script"].apply(lambda x: airtable_script.get(x)['fields']['locuteur'])
  df_ec_script["seq"] = df_ec_script["ec_script"].apply(lambda x: "{:02d}".format(airtable_script.get(x)['fields']['uid']))
  df_ec_script["ID"] = df_ec_script[["EC_Key","seq"]].apply(lambda x: x[0] + "-FRA-001_" + x[1], axis=1)
  df_ec_script["AudioFR"] = df_ec_script["ID"].apply(lambda x: "Files_Files_/assets/audio/" + x + ".mp3")

  df_ec_script["Sens"] = df_ec_script["Locuteur"].apply(lambda x: "A" if x == "L’orthoptiste" else "B")

  df_ec_script.drop(["uid","ec_name","ec_script","seq"], axis=1, inplace=True)

  frames =[]
  frames.append(df_ec_script_content)
  frames.append(df_ec_script)
  df_ec_script_update = pd.concat(frames)
  
  # on enregistre dans GSheet
  try:
    t_corpus = wb.worksheet("t_ec_content_fr")
    wb.del_worksheet(t_corpus)
  except:
    print ("Onglet inexistant !")
    pass
  wb.add_worksheet("t_ec_content_fr", 1, 1)
  export_sheet = wb.worksheet("t_ec_content_fr")
  set_with_dataframe(export_sheet, df_ec_script_update)  

# Lemmatization et Tokeniszation

In [94]:
from transformers import pipeline
# https://blog.huawei.com/2022/02/01/speaking-your-language-transformer-machine-translation/
# https://towardsdatascience.com/keyword-extraction-with-bert-724efca412ea


In [74]:
from deep_translator import GoogleTranslator

In [84]:
classifier = pipeline("text-classification")
s = "Si la douleur descend dans la nuque, c'est normal car le centre de vision est situé ici derrière la nuque."

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [85]:
t = GoogleTranslator(source="fr", target="en").translate(s)

In [86]:
t

"If the pain goes down the neck, it's normal because the center of vision is located here behind the neck."

In [77]:
outputs = classifier (t)

In [78]:
outputs

[{'label': 'POSITIVE', 'score': 0.9994404911994934}]

In [70]:
reader = pipeline("question-answering")

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/473 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/261M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [90]:
question = "Is it normal if the pain goes down the neck ?"

In [91]:
outputs = reader(question=question, context=t)

In [93]:
model_name = 'qanastek/51-languages-classifier'

In [92]:
outputs

{'score': 0.5011228919029236, 'start': 32, 'end': 43, 'answer': "it's normal"}

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = set(nltk.corpus.stopwords.words('english')) | set(nltk.corpus.stopwords.words('french'))

s="Si la douleur descend dans la nuque, c'est normal car le centre de vision est situé ici derrière la nuque."

from sklearn.feature_extraction.text import CountVectorizer

n_gram_range = (1, 1)
stop_words = stopwords.words('english') + stopwords.words('french')

# Extract candidate words/phrases
count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([s])
candidates = count.get_feature_names()

candidates

Language identification 
[Hugging Face](https://huggingface.co/qanastek/51-languages-classifier)

#FIN pour l'instant

In [ ]:
idx

Int64Index([24], dtype='int64')

In [ ]:
airtable_service.get('reczF0C8BTkRcbzxL')

{'id': 'reczF0C8BTkRcbzxL',
 'createdTime': '2022-09-16T07:05:38.000Z',
 'fields': {'Etudes de cas': ['rec4OsPEdfAgkjvw9'],
  'Site': 'https://www.chu-nantes.fr/ophtalmologie-4',
  'Résumé': "Le service d'ophtalmologie du CHU de Nantes assure à la fois des missions de soins, de recherche et d'enseignement.\nLes soins prodigués concernent les corrections optiques et chirurgicales des troubles réfractifs et la prise en charge de l'ensemble des pathologies oculaires médicales et chirurgicales de l'œil et de la sphère péri-oculaire.\nNous assurons annuellement environ 30.000 consultations et 3.000 interventions.\nNous sommes dotés des moyens d'investigation et de traitement les plus modernes.\nDomaines d'expertises\xa0\n- Strabismes et ophtalmo-pédiatrie\n- Pathologie et chirurgie des paupières, voies lacrymales et orbite\xa0\n- Pathologie et chirurgie de la cornée et de la conjonctive, kératocône, greffes\n- Chirurgie réfractive (chirurgie de la myopie, de l'hypermétropie et de l'astigmat

In [ ]:
record['fields']['Résumé']

"Le service d'ophtalmologie du CHU de Nantes assure à la fois des missions de soins, de recherche et d'enseignement.\nLes soins prodigués concernent les corrections optiques et chirurgicales des troubles réfractifs et la prise en charge de l'ensemble des pathologies oculaires médicales et chirurgicales de l'œil et de la sphère péri-oculaire.\nNous assurons annuellement environ 30.000 consultations et 3.000 interventions.\nNous sommes dotés des moyens d'investigation et de traitement les plus modernes.\nDomaines d'expertises\xa0\n- Strabismes et ophtalmo-pédiatrie\n- Pathologie et chirurgie des paupières, voies lacrymales et orbite\xa0\n- Pathologie et chirurgie de la cornée et de la conjonctive, kératocône, greffes\n- Chirurgie réfractive (chirurgie de la myopie, de l'hypermétropie et de l'astigmatisme)\n- Chirurgie de la cataracte\xa0\xa0\xa0\xa0\n- Glaucomes\xa0\n- Chirurgie de la rétine et du vitré\xa0\xa0\xa0\xa0\xa0\n- Inflammation oculaire (uvéites)\n- Greffes de cornée\xa0\xa0\x

In [ ]:
df_vk_ec.dtypes

Date             object
Statut           object
Nom              object
CR               object
uid               int64
Contexte         object
ID               object
Thème            object
Domaine          object
Présentation     object
Interprète       object
Lieu             object
Language         object
Language_code    object
Cover            object
Audio1           object
Audio2           object
dtype: object

In [ ]:
df_ec_meta.dtypes

ID               object
Nom              object
Domaine          object
Thème            object
Language_code    object
Language         object
Date             object
Lieu             object
Interprète       object
Statut           object
Cover            object
CR               object
Audio1           object
Audio2           object
Présentation     object
Contexte         object
dtype: object

In [ ]:
df_vk_ec.dtypes

ec_service               object
Notes                    object
Date                     object
Statut                   object
ec_location              object
Nom                      object
ec_interpret             object
Domaine                  object
interpret_nom_complet    object
ec_langue                object
Lieu                     object
CR                       object
uid                       int64
Attachments              object
ec_contexte              object
ec_script                object
Présentation             object
ID                       object
Thème                    object
Language_code            object
Language                 object
Interprète               object
Cover                    object
Audio1                   object
Audio2                   object
dtype: object

In [ ]:
df_ec_meta

,ID,Nom,Domaine,Thème,Language_code,Language,Date,Lieu,Interprète,Statut,Cover,CR,Audio1,Audio2,Présentation,Contexte
0,EC01,Etude de cas n°1,Médical,Hepato-Gastroentérologie,rus,Russe,16 décembre 2021 14:15,CHU,Nina,Réalisé,Files_Files_/assets/images/EC01.png,https://drive.google.com/open?authuser=0&id=1c...,,,,
1,EC02,Etude de cas n°2,Médical,Vaccination,rom,Roumain,4 janvier 2022 10:30,PASS Consultation Jean-Guillon,Alexandra,Réalisé,Files_Files_/assets/images/EC02.png,https://drive.google.com/open?authuser=0&id=1P...,,,,
2,EC03,Etude de cas n°3,Médical,PASS,rus,Russe,7 janvier 2022 08:45,PASS Consultation Jean-Guillon\nBâtiment Le To...,Nina,Réalisé,Files_Files_/assets/images/EC03.png,https://drive.google.com/open?authuser=0&id=1Q...,,,La permanence d'accès aux soins de santé (Pass...,La patient est une femme de nationalité russe....
3,EC04,Etude de cas n°4,Médical,PASS,rom,Roumain,28 décembre 2021 17:28,PASS Consultation Jean-Guillon\nBâtiment Le To...,,Réalisé,Files_Files_/assets/images/EC04.png,https://drive.google.com/open?authuser=0&id=1x...,,,,
4,EC05,Etude de cas n°5,Médical,Vaccination,rom,Roumain,28 décembre 2021 17:31,PASS Consultation Jean-Guillon,,Réalisé,Files_Files_/assets/images/EC05.png,https://drive.google.com/open?authuser=0&id=1S...,,,,
5,EC06,Etude de cas n°6,Médical,Pédiatrie,tur,Turc,26 janvier 2022 11:30,CHU,Gulseren,Réalisé,Files_Files_/assets/images/EC06.png,https://docs.google.com/document/d/1cwEgb2QoZq...,,,,
6,EC07,Etude de cas n°7,Médical,PASS,tur,Turc,2 février 2022 14:15,PASS Consultation Jean-Guillon,Canan,Réalisé,Files_Files_/assets/images/EC07.png,https://docs.google.com/document/d/1oLOmvPwCh5...,,,,
7,EC08,Etude de cas n°8,Médical,Infectiologie,ams,Arabe (Moderne Standard),15 février 2022 09:30,CHU,Wafa,Réalisé,Files_Files_/assets/images/EC08.png,https://docs.google.com/document/d/1NtTtUkIVJv...,,,,
8,EC09,Etude de cas n°9,Médical,Orthopédie,dri,Dari,4 mars 2022 14:24,CHU,Massouda,Réalisé,Files_Files_/assets/images/EC09.png,https://docs.google.com/document/d/1x4vKkLryaP...,,,,
9,EC10,Etude de cas n°10,Médical,Hémophilie,hun,Hongrois,22 mars 2022 11:30,CHU,Annamaria,Réalisé,Files_Files_/assets/images/EC10.png,https://docs.google.com/document/d/1nWFkcB1n4s...,,,,


In [ ]:
df_vk_ec.loc[~df_vk_ec["ID"].isin(df_ec_meta["ID"])]

,Date,Statut,Nom,CR,Contexte,ID,Thème,Domaine,Présentation,Interprète,Lieu,Language,Language_code,Cover,Audio1,Audio2
24,2022-06-16T08:21:00.000Z,Réalisé,Etude de cas n°21,https://docs.google.com/document/d/1yZHKR-Ff87...,"Femme originiaire du Burundi, ayant cassé ses ...",EC21,Ophtalmologie,Médical,Le service d'ophtalmologie du CHU de Nantes as...,Aline Uwimana,Hôtel-Dieu\n1 place Alexis-Ricordeau\n6e étage...,Kirundi,rnd,Files_Files_/assets/images/EC21.png,<NA>,<NA>


In [ ]:
df_vk_ec.dtypes

ec_service               object
Notes                    object
ec_date                  object
ec_status                object
ec_location              object
ec_name                  object
ec_interpret             object
ec_domaine               object
interpret_nom_complet    object
ec_langue                object
location_adress          object
ec_doc_url               object
uid                       int64
Attachments              object
ec_contexte              object
ec_script                object
service_resume           object
ID                       object
dtype: object

In [ ]:
vk_filter = ["rus"]
# on itère sur les langues choisies pour le run
for FILTER_LANG in vk_filter:
  # on itère sur le panier de phrases en français
  for index, row in df_ec_fr.iterrows():
    key_a = row["ID"]
    key_b = key_a[:5] + FILTER_LANG.upper() + "-" + key_a[9:]
    # on cherche si une traduction IA est déjà faite pour cette langue
    

    print (key_b)

In [ ]:
# concaténation des dataframes
frames =[]
frames.append(df_ec_nlp)

# on fait une passe par langue et par langue pivot
vk_filter = ["eng"]
for FILTER_LANG in vk_filter:
  for dic in dict_lang_vendor:
    language = dic["language"]
    iso_code = dic["iso-code"]
    # Si la langue est celle du filtre ?
    if language == FILTER_LANG:
      # Pour chaque code vendor qui gère la langue 
      for vc in dic["vendor-code"]:
        df_return = call_nlp_language_vendor (FILTER_LANG, iso_code, vc, FILTER_PIVOT)
        frames.append(df_return)

,ID,PhraseFR,AudioFR,EC_Key,Locuteur,Sens,translation_pivot
0,EC03-FRA-001_01,Bonjour je suis le Médecin S. et je suis acco...,Files_Files_/assets/audio/EC03-FRA-001_01.mp3,EC03,Médecin,A,Bonjour je suis le Médecin S. et je suis acco...
1,EC03-FRA-001_02,Comme c’est la première fois qu’on se rencontr...,Files_Files_/assets/audio/EC03-FRA-001_02.mp3,EC03,Médecin,A,Comme c’est la première fois qu’on se rencontr...
2,EC03-FRA-001_03,Vous êtes originaire de Géorgie ? de Russie ?,Files_Files_/assets/audio/EC03-FRA-001_03.mp3,EC03,Médecin,A,Vous êtes originaire de Géorgie ? de Russie ?
3,EC03-FRA-001_04,De Russie,Files_Files_/assets/audio/EC03-FRA-001_04.mp3,EC03,Patient,B,De Russie
4,EC03-FRA-001_05,Quand êtes-vous arrivée en France ?,Files_Files_/assets/audio/EC03-FRA-001_05.mp3,EC03,Médecin,A,Quand êtes-vous arrivée en France ?
...,...,...,...,...,...,...,...
95,EC01-FRA-001_47,C’est quoi comme type de douleur ? Poignard ? ...,,EC01,Médecin,A,C’est quoi comme type de douleur ? Poignard ? ...
96,EC01-FRA-001_48,C’est comme une douleur de faim en continu,,EC01,Médecin,A,C’est comme une douleur de faim en continu
97,EC01-FRA-001_49,Ce pourrait être un ulcère de l’estomac ou de ...,,EC01,Médecin,A,Ce pourrait être un ulcère de l’estomac ou de ...
98,EC01-FRA-001_50,Avez-vous des questions ?,,EC01,Médecin,A,Avez-vous des questions ?


# FIN IA

In [ ]:
# concaténation des dataframes
frames =[]
frames.append(df_corpus_nlp)
frames.append(df_return)
#frames.append(df_deepl)
df_langdeck_ai = pd.concat(frames)
# on uniformise le type
df_langdeck_ai['translation_ai_source'] = df_langdeck_ai['translation_ai_source'].astype(str)
# on supprime les doublons
df_langdeck_ai = df_langdeck_ai.drop_duplicates(subset=['uid'], keep='last')
# on supprime les codes ai == 0
df_langdeck_ai = df_langdeck_ai.loc[df_langdeck_ai["translation_ai_source"]!="0"]


Onglet inexistant !


APIError: ignored

In [ ]:
# dataframe temporaire pour chaque appel IA
ai = "12"
df_langdeck_ai_temp = df_langdeck.copy()

# ajout d'un uid par concaténation des 3 clés
df_langdeck_ai_temp["uid"] = df_langdeck_ai_temp["hash"].map(str) + ai 
df_langdeck_ai_temp = df_langdeck_ai_temp [["uid","hash","maitre","expression","language","translation","translation_pivot","translation_state","translation_ai_source"]]
df_langdeck_ai_temp["translation_ai"]=""

# on exclut les entrées déjà renseignées par l'iA
df_langdeck_ai_temp = df_langdeck_ai_temp.loc[(~df_langdeck_ai_temp["uid"].isin(df_corpus_nlp["uid"])) & (df_langdeck_ai_temp["language"]==filter)]
# on appelle la traduction IA si :
# - le uid n'existe pas dans la DF IA df_corpus_nlp
# - le filtre est positionné


In [ ]:
df_langdeck_ai_temp

In [ ]:
# on appelle la traduction IA si :
# - le uid n'existe pas dans la DF IA df_corpus_nlp
# - le filtre est positionné
if len(df_langdeck_ai_temp) > 0:
  from deep_translator import GoogleTranslator
  df_langdeck_ai_temp[["translation_ai","translation_state","translation_ai_source"]] = df_langdeck_ai_temp[["translation","translation_pivot","language","translation_state"]].apply(
      lambda x:translation_google(x[1],x[2], filter, ai), 
      axis=1, 
      result_type="expand")
df_google = df_langdeck_ai_temp.copy()

## Appel DeepL

In [ ]:
# dataframe temporaire pour chaque appel IA
df_langdeck_ai_temp = df_langdeck.copy()
ai = "14"
# ajout d'un uid par concaténation des 3 clés
df_langdeck_ai_temp["uid"] = df_langdeck_ai_temp["hash"].map(str) + ai 
df_langdeck_ai_temp = df_langdeck_ai_temp [["uid","hash","maitre","expression","language","translation","translation_pivot","translation_state","translation_ai_source"]]
df_langdeck_ai_temp["translation_ai"]=""

# on exclut les entrées déjà renseignées par l'iA
df_langdeck_ai_temp = df_langdeck_ai_temp.loc[(~df_langdeck_ai_temp["uid"].isin(df_corpus_nlp["uid"])) & (df_langdeck_ai_temp["language"]==filter)]
# on appelle la traduction IA si :
# - le uid n'existe pas dans la DF IA df_corpus_nlp
# - le filtre est positionné


In [ ]:
df_langdeck_ai_temp

In [ ]:
from deep_translator import DeeplTranslator
df_langdeck_ai_temp[["translation_ai","translation_state","translation_ai_source"]] = df_langdeck_ai_temp[["translation","translation_pivot","language","translation_state"]].apply(
    lambda x:translation_deepl(x[1],x[2], filter, ai), 
    axis=1, 
    result_type="expand")
df_deepl = df_langdeck_ai_temp.copy()

## On sauvegarde

In [ ]:
from deep_translator import PonsTranslator
filter = "hun"
ai = 13
df_langdeck_ai_temp[["translation_ai","translation_state","translation_ai_source"]] = df_langdeck_ai_temp[["translation","translation_pivot","language","translation_state"]].apply(
    lambda x:translation_pons(x[1],x[2], filter, ai), 
    axis=1, 
    result_type="expand")
df_pons = df_langdeck_ai_temp.copy()

### Ajout du dataset Google dans le dataset global IA

In [ ]:
frames =[]
frames.append(df_google)
frames.append(df_pons)
df_langdeck_ai = pd.concat(frames)

In [ ]:
# ajout d'un uid par concaténation des 3 clés
df_langdeck_ai["uid"] = df_langdeck_ai["hash"].map(str) + df_langdeck_ai["translation_ai_source"].map(str) 

In [ ]:
df_langdeck_ai.loc[(df_langdeck_ai["language"]=="hun") & (df_langdeck_ai["hash"]=="30579682hun")]

NameError: ignored

In [ ]:
def fetch_deep_translator(hashid, language, vendor-name_code, row):
  iso_code = iso_codes[language]
  ai_dict = next(filter(lambda x: x.get('vendor-code') == vendor-name_code,deep_translator_vendor-name_codes))
  vendor-name = ai_dict["vendor-name"]
  pivot = ai_dict["pivot"]
  if pivot == "en":
    pivot_expr = row["translation_pivot"]
  else:
    pivot_expr = row["expression"]
  message = "Appel de l'IA Deep " + vendor-name + " pour la langue " + iso_code + " pour l'expression " + pivot_expr


  df_langdeck_ai_temp[["translation_ai","translation_state","translation_ai_source"]] = df_langdeck_ai_temp[["translation","translation_pivot","language","translation_state"]].apply(
      lambda x:translation_google(x[1],x[2], filter, ai), 
      axis=1, 
      result_type="expand")

  return message

In [ ]:
hash_ai = "7c15835bukr12"
len(df_corpus_nlp.loc[df_corpus_nlp["hash_ai"]==hash_ai])

0

In [ ]:
filtre = "ams"
vc = "11"
for key, value in df_langdeck.iterrows():
  if value["language"]==filtre:
    # on recherche la clé hash+ia dana la table deep
    hash_ai = value["hash"]+vc
    found = df_corpus_nlp.loc[df_corpus_nlp["hash_ai"]==hash_ai]
    # pas trouvé, on passe la ligne pour appel ia
    if len(found)==0:
      print ("Recherche de " +hash_ai)
      print (fetch_deep_translator(value["hash"], value["language"], vc, value))


In [ ]:

translated = GoogleTranslator(source='auto', target='de').translate("keep it up, you are awesome")  # output -> Weiter so, du bist großartig

NameError: ignored

Google Translator

Pons

In [ ]:
'''
filter = "rou"
ai = 11
# algorithme :
# si state==1(pending) ou 2(validé) => pas d'appel IA
# si state==0 => appel ia
df_langdeck[["translation","translation_state","translation_ai_source"]] = df_langdeck[["translation","translation_pivot","language","translation_state"]].apply(
    lambda x:fetch_deep_tlr_txtai(x[1],x[2], filter, ai) 
    if x[3]==0 
    else (x[0],x[3],0), 
    axis=1, 
    result_type="expand")
'''

'\nfilter = "rou"\nai = 11\n# algorithme :\n# si state==1(pending) ou 2(validé) => pas d\'appel IA\n# si state==0 => appel ia\ndf_langdeck[["translation","translation_state","translation_ai_source"]] = df_langdeck[["translation","translation_pivot","language","translation_state"]].apply(\n    lambda x:fetch_translation_ia_txtai(x[1],x[2], filter, ai) \n    if x[3]==0 \n    else (x[0],x[3],0), \n    axis=1, \n    result_type="expand")\n'

In [ ]:
df_langdeck_ai_temp["hash_ai"]=df_langdeck_ai_temp[["hash","translation_ai_source"]].apply(lambda x:x[0]+str(x[1]), axis=1)

In [ ]:
df_langdeck_ai_temp = df_langdeck_ai_temp.drop(df_langdeck_ai_temp[df_langdeck_ai_temp["translation_ai_source"]==0].index)

In [ ]:
frames = [df_langdeck_ai, df_langdeck_ai_temp]
df_langdeck_ai = pd.concat(frames)

In [ ]:
df_langdeck_ai.loc[df_langdeck_ai["translation_ai_source"]==13]

,hash,maitre,expression,language,translation,translation_pivot,translation_state,translation_ai_source,translation_ai,hash_ai
0,f9bc13efams,Médicament,À quel moment de la journée vous prenez le méd...,ams,في أي وقت من اليوم تتناول الدواء؟,At what time of day do you take the medicine?,2,13,أثْناءَ,f9bc13efams13
1,6e65dde7ams,Abcès,Abcès,ams,خُرّاج,Abscesses,2,13,خُرَاج <خُرَاجات>,6e65dde7ams13
2,80cbaceaams,Abdomen,Abdomen,ams,البطن,Abdomen,2,13,البَطْن,80cbaceaams13
3,95c4a863ams,Fébrile,Accès fébrile concomitant chez un autre membre...,ams,نوبة حمى مزامنة عند فرد اخر من الأسرة,Concomitant febrile attack in an other family ...,0,13,,95c4a863ams13
4,30579682ams,Accident,Accident,ams,حادث,Accident,2,13,مُصَادَفَة,30579682ams13
...,...,...,...,...,...,...,...,...,...,...
5529,e352c9c7ams,Sinusite,Sinusite,ams,أضف تعليقًا,Sine [K],2,13,جيب الزاية,e352c9c7ams13
5530,be431a55ams,Ulcère,Ulcère,ams,Ulcer,Ulcer,2,13,قَرْحَة <قَرْحات>,be431a55ams13
5531,7019983eams,Ulcère,Ulcère de l'estomac,ams,مُلْسَكَر,Stomach ulcer,2,13,قَرْحَةُ المَعِدَة,7019983eams13
5532,a2e70804ams,Ulcère,Ulcère de l'intestin grêle,ams,أُجُزأة صغيرة من الأم,Ulcer of the small intestine,2,13,المِعَى الدَّقيق,a2e70804ams13


In [ ]:
try:
  t_corpus_all_entries = wb.worksheet("t_translate_deep")
  wb.del_worksheet(t_corpus_all_entries)
except:
  print ("Onglet inexistant !")

wb.add_worksheet("t_translate_deep", 1, 1)
export_sheet = wb.worksheet("t_translate_deep")
set_with_dataframe(export_sheet, df_langdeck_ai)

Onglet inexistant !


In [ ]:

#df_langdeck["translation_ai_source"] = np.nan
df_langdeck["translation_score"] = df_langdeck["translation_state"].apply(lambda x: (1.0 if x==1 else 0.5 if x==2 else 0.0))
df_langdeck["translation_state_txt"] = df_langdeck["translation_state"].apply(lambda x: ("OK" if x==1 else "Pending" if x==2 else "Reject"))

In [ ]:
try:
  t_corpus_all_entries = wb.worksheet("t_corpus_all_entries")
  wb.del_worksheet(t_corpus_all_entries)
except:
  print ("Onglet inexistant !")

wb.add_worksheet("t_corpus_all_entries", 1, 1)
export_sheet = wb.worksheet("t_corpus_all_entries")
set_with_dataframe(export_sheet, df_langdeck)

# Vue Lexique (deprecated)

In [ ]:
t_lexique = wb.worksheet("t_lexicon")
# on recharge avec les nouvelles données

### nouvelles colonnes Thèmes

In [ ]:
# on crée une liste des valeurs uniques des 3 colonnes de thèmes
frames = [df_langdeck["theme_1"],df_langdeck["theme_2"],df_langdeck["theme_3"]]
vk_theme = list(set(pd.concat(frames).to_list()))
vk_theme.remove('')

In [ ]:
# on ajoute les colonnes à partir de la liste et on valorise T/F
df_lexique = df_langdeck[["hash","maitre","ordre","expression","language","language_fulltext","theme_1","theme_2","theme_3"]].copy()
df_lexique = pd.concat([df_lexique, pd.DataFrame(columns = vk_theme)])
for item in vk_theme:
  df_lexique[item] = df_lexique[["theme_1","theme_2","theme_3"]].apply(lambda x:True if (x[0]==item or x[1]==item or x[2]==item) else False, axis=1)

In [ ]:
frames=[]
df=pd.DataFrame()
frames.append(df)
# pour chaque thème on ajoute les expressions trouvées
for item in vk_theme:
  df_temp = df_lexique.loc[df_lexique[item] == True]
  df_temp.insert(0,'theme', item)
  df_temp.insert(0,'section', df_temp["ordre"].apply(lambda x:("Vocabulaire" if x==1 else "Expression" if x==2 else "Phrase" if x>=3 else "NONE")))
  df_temp.insert(0,'theme_hash', df_temp["hash"].apply(lambda x:"".join(item.split())+x))
  frames.append(df_temp)
  # on concatène le tout dans un new dataframe
  df_langdeck_theme = pd.concat(frames)

In [ ]:
len(df_langdeck_theme)

7280

In [ ]:
df_langdeck_theme.iloc[1]

theme_hash                  Médecinegénérale6e65dde7ams
section                                     Vocabulaire
theme                                 Médecine générale
hash                                        6e65dde7ams
maitre                                            Abcès
ordre                                               1.0
expression                                        Abcès
language                                            ams
language_fulltext                                 Arabe
theme_1                               Médecine générale
theme_2                                                
theme_3                                                
Médecine générale                                  True
Etat civil                                        False
Vaccination                                       False
Hépato-Gastroentérologie                          False
Douleur                                           False
Nourriture                                      

In [ ]:
try:
  tbl = wb.worksheet("t_voc_synth")
  wb.del_worksheet(tbl)
except:
  print ("Onglet inexistant !")

wb.add_worksheet("t_voc_synth", 1, 1)
export_sheet = wb.worksheet("t_voc_synth")
set_with_dataframe(export_sheet, df_langdeck)

In [ ]:
try:
  t_lexique = wb.worksheet("t_lexicon")
  wb.del_worksheet(t_lexique)
except:
  print ("Onglet inexistant !")

wb.add_worksheet("t_lexicon", 1, 1)
export_sheet = wb.worksheet("t_lexicon")
set_with_dataframe(export_sheet, df_langdeck_theme)

# FIN pour l'instant

# Appel IA Google Translate

In [ ]:
# txtAI
translation = translate("Forearm", "ar")
translation

'لـ مُرْرْم'

In [ ]:
def txtai_alpha (input, iso_code):
  try:
    translation = translate(input, "ar")
  except Exception as e:
    translation = ""
    pass
  return translation, 2


In [ ]:
df_txtai = df_langdeck.loc[(df_langdeck["language"]=="ams") & (df_langdeck["translation_state"]==0) & (df_langdeck["idx"]=="B")]

In [ ]:
df_txtai

,hash,uid,maitre,ordre,expression,idx,timestamp,language,translation,language_fulltext,...,translation_state,Corps humain,Etat civil,Hépato-Gastroentérologie,Examen du patient,Vaccination,Maladie,Dossier du patient,Douleur,Médecine générale
56,f3bff76cams,f3bff76c,Bonjour,1.0,Bonjour,B,2022-03-31 14:33:26,ams,,Arabe,...,0.0,False,False,False,False,False,False,True,False,True
58,b80ffb21ams,b80ffb21,Bouche,1.0,Bouche,B,2022-03-31 15:25:18,ams,,Arabe,...,0.0,True,False,False,False,False,False,False,False,False


In [ ]:
df_txtai[["translation","translation_state"]]=df_txtai[["expression","translation_state"]].apply(lambda x: txtai_alpha(x[0], "ar") if x[1]==0 else "", axis=1, result_type="expand")

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [ ]:
df_txtai

,hash,uid,maitre,ordre,expression,idx,timestamp,language,translation,language_fulltext,...,translation_state,Corps humain,Etat civil,Hépato-Gastroentérologie,Examen du patient,Vaccination,Maladie,Dossier du patient,Douleur,Médecine générale
56,f3bff76cams,f3bff76c,Bonjour,1.0,Bonjour,B,2022-03-31 14:33:26,ams,مرحباً، مرحباً، مرحباً، مرحباً، مرحباً، مرحباً...,Arabe,...,2,False,False,False,False,False,False,True,False,True
58,b80ffb21ams,b80ffb21,Bouche,1.0,Bouche,B,2022-03-31 15:25:18,ams,لاشتششششششششششششش,Arabe,...,2,True,False,False,False,False,False,False,False,False


## Mise à jour inutile dans Langdeck !!

In [ ]:
df_airtable_for_langdeck = get_df_airtable_for_langdeck()
df_airtable_for_langdeck["last_updated"] = df_airtable_for_langdeck[["hash","unixtime"]].apply(lambda x:compare_langdeck_time(x[0],x[1]), axis=1) 
df_newly_updated_in_airtable = df_airtable_for_langdeck.loc[df_airtable_for_langdeck["last_updated"]]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  from ipykernel import kernelapp as app


### Récupération des audios

In [ ]:
df_newly_updated_in_airtable


,uid,maitre,ordre,expression,idx,timestamp,language,translation,hash,language_fulltext,unixtime,last_updated
78,8745562b,Comment,3,Comment allez-vous ?,C,2022-04-11 13:41:03,ams,NaN,8745562bams,Arabe,1649684463000000000,True
187,2bd46073,Main,1,Main,M,2022-04-11 13:41:01,ams,NaN,2bd46073ams,Arabe,1649684461000000000,True
201,80e0c323,Médicament,1,Médicament(s),M,2022-04-11 13:41:02,ams,تَورُّم,80e0c323ams,Arabe,1649684462000000000,True
257,ba312208,Âge,3,Quel âge avez-vous ?,A,2022-04-11 13:41:02,ams,ما سبب زيارتك؟,ba312208ams,Arabe,1649684462000000000,True
529,2bd46073,Main,1,Main,M,2022-04-11 13:41:01,eng,Hand,2bd46073eng,Anglais,1649684461000000000,True
...,...,...,...,...,...,...,...,...,...,...,...,...
5045,ba312208,Âge,3,Quel âge avez-vous ?,A,2022-04-11 13:41:02,pol,Ile masz lat ?,ba312208pol,Polonais,1649684462000000000,True
5208,8745562b,Comment,3,Comment allez-vous ?,C,2022-04-11 13:41:03,tig,NaN,8745562btig,Tigrinya,1649684463000000000,True
5317,2bd46073,Main,1,Main,M,2022-04-11 13:41:01,tig,NaN,2bd46073tig,Tigrinya,1649684461000000000,True
5331,80e0c323,Médicament,1,Médicament(s),M,2022-04-11 13:41:02,tig,ከኒና,80e0c323tig,Tigrinya,1649684462000000000,True


## Mise à jour dans Langdeck

In [ ]:
l = len(df_newly_updated_in_airtable)
# on parcoure la table cible
for i in range (2, l):
  # un enregistrement
  tuple_list = t_corpus.row_values(i)
  # on recherche la clé dans le dataframe source
  t_ = df_airtable_for_langdeck.loc[(df_airtable_for_langdeck['uid']==tuple_list[0]) & (df_airtable_for_langdeck['language']==tuple_list[6])]
  # verification du hash
  if len(t_)==1:
    t_corpus.update_cell(i, 9, t_["hash"].values[0])
  # comparaison des ts (A=AT;B=GG)
  ts_a = pd.to_datetime(t_["timestamp"].values[0], "%Y-%m-%d %H:%M:%S")
  ts_b = datetime.datetime.strptime(tuple_list[5], "%Y-%m-%d %H:%M:%S")
  # print (t_["hash"].values[0] + ": Airtable ts " + str(ts_a) + " GG ts " + str(ts_b))
  if ts_a > ts_b:
    # cas Airtable + récent => update dans Langdeck
    t_corpus.update_cell(i, 2, t_["maitre"].values[0])
    t_corpus.update_cell(i, 3, int(t_["ordre"].values[0]))
    t_corpus.update_cell(i, 4, t_["expression"].values[0])
    t_corpus.update_cell(i, 5, t_["idx"].values[0])
    if not pd.isnull(t_["translation"].values[0]):
      t_corpus.update_cell(i, 8, t_["translation"].values[0])    
  elif ts_b > ts_a:
    # cas Langdeck + récent => update translation dans AT uniqumeent !
    record = airtable.match('UID', tuple_list[0])
    if bool(record) is True:
      if not pd.isnull(tuple_list[7]):
        aDict = {}
        aDict[tuple_list[6]] = tuple_list[7]
        airtable.update(record['id'], aDict)
        print (record['id'])


    


In [ ]:
frames = [df_airtable_for_langdeck, df_langdeck]
result = pd.concat(frames, keys=["hash"])
result.drop_duplicates(subset=['hash'], inplace=True, keep='last')

In [ ]:
result

uid      maitre  ordre                                         expression idx           timestamp language                                        translation         hash language_fulltext
hash 0     f9bc13ef  Médicament      3  À quel moment de la journée vous prenez le méd...   M 2022-04-01 08:15:21      ams                  في أي وقت من اليوم تتناول الدواء؟  f9bc13efams             Arabe
     1     6e65dde7       Abcès      1                                              Abcès   A 2022-03-29 15:58:04      ams                                             خُرّاج  6e65dde7ams             Arabe
     2     80cbacea     Abdomen      1                                            Abdomen   A 2022-04-07 08:42:16      ams                                              البطن  80cbaceaams             Arabe
     3     95c4a863     Fébrile      3  Accès fébrile concomitant chez un autre membre...   F 2022-04-01 08:27:19      ams              نوبة حمى مزامنة عند فرد اخر من الأسرة  95c4a863ams             Arabe
     4     30579682    Accident      1                                           Accident   A 2022-03-29 15:58:05      ams                                               حادث  30579682ams             Arabe
...             ...         ...    ...                                                ...  ..                 ...      ...                                                ...          ...               ...
     5451  d112f379       Poids      3                        Votre poids est-il stable ?   P 2022-03-31 14:33:52      tig                                                NaN  d112f379tig          Tigrinya
     5452  2344ba84   Accoucher      3  Vous avez accouché par césarienne ou voie basse ?   A 2022-03-29 16:00:33      tig                             ብኦፕረሽን ወይ ብኖርማር ወሊድኪ ?  2344ba84tig          Tigrinya
     5453  36e66161  Contagieux      3            Y a-t-il des cas contagieux à l'école ?   C 2022-03-29 16:00:33      tig                                 አብ ትምህርቲ ዲዩ ለጊብካ ?  36e66161tig          Tigrinya
     5454  c32f5fd9   Chronique      3  Y a-t-il des maladies chroniques dans votre fa...   C 2022-04-01 08:23:46      tig                      ካብ በተሰብካ ሕዱር ሕማሕ ዘለዎ አለካ ዲዩ ?  c32f5fd9tig          Tigrinya
     5455  d3dba8e4        Zona      1                                               Zona   Z 2022-03-29 16:00:34      tig  ዞና ዝብሃል ሕማም ኣብ ቖርበት ማለት ብደገ ዝቖስል ብርቱዕ ቓንዛ ዘለዎ ...  d3dba8e4tig          Tigrinya

[5456 rows x 10 columns]

In [ ]:
df_airtable_for_langdeck[(df_airtable_for_langdeck['hash'].isin(df_langdeck['hash']) & ~df_airtable_for_langdeck['timestamp'].isin(df_langdeck['timestamp']) )]

,uid,maitre,ordre,expression,idx,timestamp,language,translation,hash,language_fulltext
2,80cbacea,Abdomen,1,Abdomen,A,2022-04-07 08:42:16,ams,البطن,80cbaceaams,Arabe


In [ ]:
set_with_dataframe(t_corpus, result)

END

### Vocabulaire avec groupement :


1.   Index alpha
2.   Mot clé
3.   Ordre



In [ ]:
vk_voc = airtable.get_all(view='Vocabulary', sort=['idx_cor','Maitre','Ordre'])
df_voc = pd.DataFrame.from_records((r['fields'] for r in vk_voc))
df_voc = df_voc[['idx_cor','Maitre','Ordre','Proposition','CATEGRAM','TYPE','Thématique']]

### Traductions avec groupement :


1.   Index alpha
2.   Mot clé
3.   Ordre

In [ ]:
df_voc_trad = airtable.get_all(view='Traductions', sort=['idx_cor','Maitre','Ordre'])
df_voc_trad = pd.DataFrame.from_records((r['fields'] for r in df_voc_trad))
df_voc_trad = df_voc_trad[['idx_cor','Maitre','Ordre','Proposition','CATEGRAM','TYPE','Thématique',
             'eng','ams','tur','rus','ukr','rou','hun','tig','alb','geo','arm',
             'dar','pst','esp','pol']]

In [ ]:
# Corpus_consolidation
uri = "https://docs.google.com/spreadsheets/d/1cSnSnnQunL-I7hpyYkIqbtuGr55unVJ3Qe8URSrjTr8/edit?usp=sharing"

wb = get_CCDB_wb(uri)
db_cc = get_CCDB_data (wb, 0)

nom_onglet = 'Lexique'
# si l'onglet existe déjà
try:
  ws = wb.worksheet(nom_onglet)
  wb.del_worksheet(ws)
except:
  print ("Onglet inexistant !")

wb.add_worksheet(nom_onglet, 1, 1)
export_sheet = wb.worksheet(nom_onglet)
set_with_dataframe(export_sheet, df_voc_trad)

Onglet inexistant !


In [ ]:
grouped = df_voc.groupby(['idx_cor','Maitre'])
'''
for name, group in grouped:
  print (name)
  print (group["Proposition"])
'''

'\nfor name, group in grouped:\n  print (name)\n  print (group["Proposition"])\n'

Recherche des UID manquants dans Airtable

In [ ]:
def airtable_get_missing_uids(df):
  return df.loc[pd.isna(df["UID"])]


Recherche dans Airtable par Séquence

In [ ]:
def airtable_get_records_by_sequence(df, sequence):
  return df.loc[df["Séquence"]==sequence]


In [ ]:
#df_vkat.loc[df_vkat["Séquence"]=="1. Présentation + antécédents"]

In [ ]:
airtable_get_missing_uids(df_vkat)

,Proposition,CATEGRAM,DEPENDANCE,idx_cor,TYPE,UID,VOCABULAIRE,Thématique,idx,Ordre,...,Updates,Séquence,MedGen,Statut,Fiche,Phrase Assoc,Définition,Corpus copy,DOMAINE,Exp Assoc
6,~ chirurgicale,[exp],[reco8iFRctnJgMJCl],I,expression,NaN,NaN,[Vaccination],I,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,Accouchement,[nm],NaN,A,vocabulaire,NaN,NaN,[Médecine générale],NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,Accoucher,[vi],NaN,A,vocabulaire,NaN,NaN,[Médecine générale],NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,Appareil ~,[exp],NaN,D,expression,NaN,NaN,[Médecine générale],NaN,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,Assurance,[nf],NaN,A,vocabulaire,NaN,Général,[Médecine générale],NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,Urine,[nf],NaN,U,vocabulaire,NaN,NaN,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
306,Vaccinal,[adj],NaN,V,vocabulaire,NaN,NaN,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
308,"Vacciné, -e",[adj],NaN,V,vocabulaire,NaN,NaN,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
317,Votre poids a-t-il augmenté ?,[phrase],[recnFsFSoE1DCjYqt],P,phrase,NaN,NaN,[Médecine générale],NaN,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Rceherche des termes présents dans Central, présents dans AIRTBL mais sans UID

In [ ]:
vk_at_trad = airtable.get_all(view='Traductions',sort='Proposition')
df_vk_at_trad = pd.DataFrame.from_records((r['fields'] for r in vk_at_trad))
df_vk_at_trad = df_vk_at_trad[["Proposition","idx_cor","UID"]]
df_vk_at_trad_notna = df_vk_at_trad.loc[pd.notna(df_vk_at_trad["UID"])]
df_vk_at_trad_na = df_vk_at_trad.loc[pd.isna(df_vk_at_trad["UID"])]

### merge côté AT : avec les UID présents 

In [ ]:
df_merged = pd.merge(df_cc_global, df_vk_at_trad_notna, how="right", left_on='uid', right_on='UID')

## On complète avec les traductions

In [ ]:
for index, row in df_merged.iterrows():
  if row["uid"] is not None:
    record = airtable.match('UID', row["uid"])
    aDict = {}
    for i in vk_languages:
      key = i["trigramme"]
      aDict[key] = row[key]
      #print (record['id'] +" " + row["eng"])
    airtable.update(record['id'], aDict)


## On complète avec les UID manquants

In [ ]:
df_vk_at_trad_na

In [ ]:
df_merged = pd.merge(df_cc_global, df_vk_at_trad_na, how="right", left_on='expression', right_on='Proposition')

In [ ]:
df_merged = df_merged.loc[pd.notna(df_merged["uid"])]

In [ ]:
for index, row in df_merged.iterrows():
  if row["uid"] is not None:
    record = airtable.match('Proposition', row["expression"])
    aDict = {}
    key = "UID"
    aDict[key] = row["uid"]
    for i in vk_languages:
      key = i["trigramme"]
      aDict[key] = row[key]
      #print (record['id'] +" " + row["eng"])
    airtable.update(record['id'], aDict)

## On vide toute la base AT dans Central Consolidation

In [ ]:
vk_at = airtable.get_all(view='Admin',sort='Proposition')
df_vkat = pd.DataFrame.from_records((r['fields'] for r in vk_at))

In [ ]:
# Corpus_consolidation
uri = "https://docs.google.com/spreadsheets/d/1cSnSnnQunL-I7hpyYkIqbtuGr55unVJ3Qe8URSrjTr8/edit?usp=sharing"

wb = get_CCDB_wb(uri)
db_cc = get_CCDB_data (wb, 0)

nom_onglet = 'Termes'
# si l'onglet existe déjà
try:
  ws = wb.worksheet(nom_onglet)
  wb.del_worksheet(ws)
except:
  print ("Onglet inexistant !")

wb.add_worksheet(nom_onglet, 1, 1)
export_sheet = wb.worksheet(nom_onglet)
set_with_dataframe(export_sheet, df_vkat)



END

In [ ]:
record = airtable.match('UID', '30579682')
fields = {'eng': 'Accident'}
airtable.update(record['id'], fields)

{'createdTime': '2021-12-10T08:47:48.000Z',
 'fields': {'CATEGRAM': ['nm'],
  'Name ID': 'recGaRQGjV5qN3QBN',
  'Proposition': 'Accident',
  'TYPE': 'vocabulaire',
  'Thématique': 'Vaccination',
  'UID': '30579682',
  'VOCABULAIRE': 'Général',
  'eng': 'Accident',
  'idx': 'A',
  'idx_cor': 'A'},
 'id': 'recGaRQGjV5qN3QBN'}

In [ ]:
record

{'createdTime': '2021-12-10T08:47:48.000Z',
 'fields': {'CATEGRAM': ['nm'],
  'Name ID': 'recGaRQGjV5qN3QBN',
  'Proposition': 'Accident',
  'TYPE': 'vocabulaire',
  'Thématique': 'Vaccination',
  'UID': '30579682',
  'VOCABULAIRE': 'Général',
  'idx': 'A',
  'idx_cor': 'A'},
 'id': 'recGaRQGjV5qN3QBN'}

In [ ]:
def airtable_table_get(uri, params, headers):
  airtable_records = []
  run = True
  while run is True:
    response = requests.get(uri, params=params, headers=headers)
    airtable_response = response.json()
    airtable_records += (airtable_response['records'])
    if 'offset' in airtable_response:
      run = True
      params = (('offset', airtable_response['offset']),)
    else:
      run = False
  return airtable_records

# Table Corpus

In [ ]:

url = "https://api.airtable.com/v0/" + base_id + "/" + table_name


### On lit tous les enregistrements, puis on met dans un dataframe

# Nouvelle section

In [ ]:
airtable_records = airtable_table_get(url, params, headers)

airtable_rows = [] 
airtable_index = []
for record in airtable_records:
  airtable_rows.append(record['fields'])
  airtable_index.append(record['id'])
df = pd.DataFrame(airtable_rows, index=airtable_index)
df = df [['Proposition','UID', 'Name ID', 'DEPENDANCE', 'idx', 'idx_cor','Statut','Thématique']]
df.sort_values(by=['Proposition'], ascending=True, inplace=True)

In [ ]:
df.head(10)

,Proposition,UID,Name ID,DEPENDANCE,idx,idx_cor,Statut,Thématique
recKbPHqDTXNPwYK0,A jeun,f397cda1,recKbPHqDTXNPwYK0,NaN,J,J,NaN,Vaccination
rec1qTnfWxjhx6SBl,AME - Aide médicale d'État,f49563a9,rec1qTnfWxjhx6SBl,NaN,A,A,NaN,Vaccination
rec0HgtD22F7TWM3W,Abcès,6e65dde7,rec0HgtD22F7TWM3W,NaN,A,A,NaN,Médecine générale
recGaRQGjV5qN3QBN,Accident,30579682,recGaRQGjV5qN3QBN,NaN,A,A,NaN,Vaccination
rec3MxDq67MC982ng,Accident cérébral,6f214e4c,rec3MxDq67MC982ng,[recGaRQGjV5qN3QBN],A,A,NaN,Vaccination
reciexfGbNKvXzQg8,Alcool,NaN,reciexfGbNKvXzQg8,NaN,A,A,NaN,Médecine générale
recfY4uKkgK8Jp8GO,Allergie,8b195cf4,recfY4uKkgK8Jp8GO,NaN,A,A,NaN,Vaccination
recSKSrXVLhDfMq6w,Allergie alimentaire,f754eebf,recSKSrXVLhDfMq6w,[recfY4uKkgK8Jp8GO],A,A,NaN,Vaccination
rectthHdPchXi1zr9,Allergie médicamenteuse,7256c169,rectthHdPchXi1zr9,"[recfY4uKkgK8Jp8GO, recOivQOmeragygVr]",A,A,NaN,Vaccination
recHNzXfvWWm0Lwp8,Allergie à un médicament,316fa7ab,recHNzXfvWWm0Lwp8,"[recfY4uKkgK8Jp8GO, recOivQOmeragygVr]",A,A,NaN,Vaccination


In [ ]:
def lookup_uid (uid):
  try:
    record = df_cc_fr.loc[df_cc_fr["uid"]==uid]["index"].item()
  except Exception as e:
    list_no_uid.append(uid)
    return uid
  if record is not None:
    return record

In [ ]:
x = df_cc_fr.loc[df_cc_fr["uid"]=="72f0e8db"]["index"].item()

In [ ]:
list_no_uid = [] # uid non présents côté central = mots à créer
df['UID'] = df['UID'].apply(lambda x: make_unique_id() if pd.isna(x) else x)
df.idx_cor = df['UID'].apply(lambda x: lookup_uid(x))

convert the dataframe to a dictionary of dictionaries.

In [ ]:
df.tail()

,Proposition,UID,Name ID,DEPENDANCE,idx,idx_cor,Statut,Thématique
recia1L6nICYg6LJ8,Zona,d3dba8e4,recia1L6nICYg6LJ8,NaN,NaN,Z,NaN,Vaccination
recVvIHdNWf80zvsG,À quelle moment de la journée vous prenez le m...,0f86d1ef,recVvIHdNWf80zvsG,[recOivQOmeragygVr],M,0f86d1ef,Ramona,Médecine générale
recWp0rajm2qKcJuF,Êtes vous vacciné (e) ?,c5df267a,recWp0rajm2qKcJuF,NaN,NaN,Ê,NaN,Médecine générale
recBY5NXGM8UtBcaX,Êtes-vous enceinte ?,7768882c,recBY5NXGM8UtBcaX,NaN,NaN,Ê,NaN,Médecine générale
reczy8Os5Hc3qZ1yA,Œdème,72f0e8db,reczy8Os5Hc3qZ1yA,NaN,NaN,Œ,NaN,Médecine générale


In [ ]:
dict_records = df.tail().to_dict(orient="index")

In [ ]:
for record_id, upload_data in dict_records.items():
  record_url = url + "/" + record_id
  upload_dict = {"records" : [{"fields" : upload_data}]}
  upload_json = json.dumps(upload_dict)
  requests.post(url, data=upload_json, headers=headers)

In [ ]:
headers

{'Authorization': 'Bearer keyrI98TIqu6mbFcf',
 'Content-Type': 'application/json'}

In [ ]:
requests.patch(url, data=upload_json, headers=headers)

<Response [422]>

In [ ]:
upload_json='{"records": [{"fields": {"Proposition": "OEDEME", "id": "reczy8Os5Hc3qZ1yA"}}]}'

In [ ]:
requests.post(record_url, data=upload_json, headers=headers)

<Response [404]>

In [ ]:
def fmt_table(wb, nom_onglet):
  # onglet
  ws = wb.worksheet(nom_onglet)

  # Format de la partie gauche
  fmt = cellFormat(
      backgroundColor=color(0.91, 0.96, 0.93),
      textFormat=textFormat(bold=False, foregroundColor=color(0,0,0), fontSize='10'),
      horizontalAlignment='LEFT'
      )
  format_cell_range(ws, 'A:', fmt)

  # format de l'entete
  fmt = cellFormat(
      backgroundColor=color(0.7725,0.8431,0.7922),
      textFormat=textFormat(bold=True, foregroundColor=color(0,0,0), fontSize='10'),
      horizontalAlignment='LEFT'
      )
  format_cell_range(ws, '1', fmt)
  
  # format de l'auto wrap
  fmt = cellFormat(
      wrapStrategy='WRAP'
      )
  format_cell_range(ws, 'B:C', fmt)

  # on gèle l'entete et les colonnes de gauche
  set_frozen(ws, rows=1, cols=1)
  set_column_width(ws, 'A:C', 500)


In [ ]:
nom_onglet = 'Liste'
# si l'onglet existe déjà
try:
  ws = wb.worksheet(nom_onglet)
  wb.del_worksheet(ws)
except:
  print ("Onglet inexistant !")

wb.add_worksheet(nom_onglet, 1, 1)
export_sheet = wb.worksheet(nom_onglet)
set_with_dataframe(export_sheet, df)
fmt_table(wb,nom_onglet)


In [ ]:
table_name = 'Structure'
url = "https://api.airtable.com/v0/" + base_id + "/" + table_name

In [ ]:
df

,Name,Situation de départ,Notes
recnI80hvydv71fs4,0.Conception,"Plus on possède, plus on est soi-même possédé....","Trouver le plaisir d’écrire. Partir d’idées, d..."
recI2pgkNq2iP52rm,1.Situation initiale,Un jour d'été où il pleut et fait un temps abo...,Une vengeance d'une femme éconduite lors d'un ...
recb2DqhB9GvfI12n,2.Personnage,Un homme (marié ?) qui a donné rendez-vous à u...,Un Dom Juan sûr de son charme et imbu de sa pe...
recNLdU01JpCnvknJ,3.Acte déclencheur,La femme lui propose un jeu. Elle est dans le ...,NaN
recRxZVhpSPD6oQXx,4.Réaction,L'homme est d'abord tenté de partir car il est...,NaN
recRr06fOM4hGGaLY,5.Péripéties,"Pendant la visite (ennuyeuse), l'homme tente d...",NaN
recvWnAYn9VNySVlB,6.Chute,Un spectacle où une partie du public est appel...,NaN
recDmZtMMNmiy7R0M,Récit,NaN,NaN
